## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

This demo uses a lot of tokens. If your quota is low, you will get throttled.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.1

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_1wZm6JLL8mryLAdomOOGkEml


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().strftime('%Y-%m-%d')
today_date
```

I called the tool named crawl with these arguments:
```
{
  "q": "things to do in Atlanta on October 3, 2024",
  "textFormat": "HTML"
}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on October 3, 2024"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=4907344191957257\u0026mkt=en-US\u0026setlang=en-US\u0026w=gk2PwJ4MGAK3mkDLUvcm_m1afK8iBTpt"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"Atlanta Fair. October 4 to November 3, 2024 Old Turner Field Atlanta. This is the region\u2019s only urban fair, taking place at Turner Field in downtown Atlanta. There\u2019s a carnival midway with rides, plenty of food, nightly entertainment, and more. It will be here for all of October. Details for Atlanta Fair. Image: Atlanta Fair","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=5012794232883316\u0026mkt=en-US\u0026setlang=en-US\u0026w=IFPPoHWFm5znjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Las nubes, D\u00E1vila 666, and Ladrones (US) The Star Community Bar, Atlanta, GA, US. Thursday 03 October 2024. Charli XCX and Troye Sivan. State Farm Arena, Atlanta, GA, US. Ziggy Marley and Marley Brothers Stephen Marley, Julian Marley, Ky-mani Marley, and Damian \u0026#39;Jr Gong\u0026#39; Marley. Cellairis Amphitheatre at Lakewood, Atlanta, GA, US.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=4726551834592006\u0026mkt=en-US\u0026setlang=en-US\u0026w=77Xe8SkyBFllYyuTqLMEVsVddRXByGfo"},{"name":"Inaugural Atlanta Art Fair Taking Place October 3 To 6 At ... - Forbes","url":"https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/","displayUrl":"https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking...","snippet":"The inaugural Atlanta Art Fair takes place October 3 through 6, 2024, at Pullman Yards five miles east of downtown. The event coincides with Atlanta\u2019s third annual Art Week.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=1352274716276\u0026mkt=en-US\u0026setlang=en-US\u0026w=zWSvhuM3lMtfGgZCrT59fpkTquctPz0s"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming ... October 4\u2013November 3, 2024 Old Turner Field The Atlanta Fair returns to Center Parc Stadium\u2014formerly Turner ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"27 Thrilling Things To Do In Atlanta This October - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-october/","displayUrl":"https://secretatlanta.co/things-to-do-october","snippet":"Every year, Zoo Atlanta hosts a trick-or-treating experience that\u2019s fun for the whole family! This year, the event will take place October 19th, 20th, 26th, and 27th, from 9 a.m. \u2013 3 p.m.! Bring the kids for trick or treating, photo ops, costumed characters, and themed activities! This is not a scary event at all!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=4673096681200628\u0026mkt=en-US\u0026setlang=en-US\u0026w=_CWggicgECxNFCQACSuK0PesDCkO7vK5","thumbnailUrl":"https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=4714289709671445\u0026mkt=en-US\u0026setlang=en-US\u0026w=uP6_4hpqL9Zfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids","url":"https://mommypoppins.com/atlanta-kids/weekend-events/best-things-to-do-in-atlanta-in-october-halloween-weekend","displayUrl":"https://mommypoppins.com/.../best-things-to-do-in-atlanta-in-october-halloween-weekend","snippet":"Families are in for a treat this October in Atlanta with a packed calendar of fun events to enjoy, including Halloween and fall festivals!. Kick off October events in Atlanta with fried food, amazing rides, and spectacular shows at the Georgia State Fair and the Cumming Country Fair and Festival.The delicious Georgia Apple Festival in Ellijay takes place across two weekends in October, so ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=1833269792522\u0026mkt=en-US\u0026setlang=en-US\u0026w=2WgIM99orhoqHTTsGTDReINgTpcgCSGt","thumbnailUrl":"https://www.bing.com/th?id=OIP.C_A0zJHZzJWMBwBVm1nvigAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Yacht Rock Revue has three Atlanta concerts in fourth quarter and a new ...","url":"https://www.ajc.com/things-to-do/georgia-entertainment-scene/yacht-rock-revue-has-three-atlanta-concerts-in-fourth-quarter-and-a-new-album/W6I5NVG3EZEYRPWKI5ZPFRR3NY/","displayUrl":"https://www.ajc.com/things-to-do/georgia-entertainment-scene/yacht-rock-revue-has...","snippet":"Yacht Rock Revue, the Atlanta-based band that has been covering the likes of Christopher Cross and Toto for 17 years, will be back at Cadence Bank Amphitheatre at Chastain on Oct. 5, 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B3%2c\u002B2024\u0026d=321411434110\u0026mkt=en-US\u0026setlang=en-US\u0026w=G1FVfOGyCmS7imqQdwfIce7BHWtjDKK9"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/october/"}
```
and got this result:
```
"Atlanta Events October 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* October\r\n\r\n# Atlanta Events October 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices. Experience the diverse and vibrant culture of Atlanta this October!\r\n\r\nStart DateOct 3, 2024ResetCancelApplyEnd DateOct 5, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 508* Oct 03 2024 7:00 PM Thu The Slaps Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $29 Avg. price ~ $33 16 tickets remaining! Tickets\r\n* Oct 03 2024 7:00 PM Thu Sepultura Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $50 Avg. price ~ $85 Available Tickets: 53 Tickets\r\n* Oct 03 2024 7:00 PM Thu La Luz Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $31 Avg. price ~ $39 49 tickets remaining! Tickets\r\n* Oct 03 2024 7:30 PM Thu The Marley Brothers Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $33 Avg. price ~ $162 Available Tickets: 1030 Tickets\r\n* Oct 03 2024 7:30 PM Thu Charli XCX \u0026 Troye Sivan State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $93 Avg. price ~ $263 Available Tickets: 432 Tickets\r\n* Oct 03 2024 7:30 PM Thu The Lion King Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $68 Avg. price ~ $138 Available Tickets: 274 Tickets\r\n* Oct 03 2024 8:00 PM Thu Johnny Marr The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $73 Avg. price ~ $136 19 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Will Burkart City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $49 Avg. price ~ $79 30 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Horse Jumper of Love Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $31 Avg. price ~ $56 16 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu MJ Lenderman and The Wind Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $29 Avg. price ~ $37 19 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Elle King The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $21 Avg. price ~ $86 Available Tickets: 429 Tickets\r\n* Oct 03 2024 8:00 PM Thu vaultboy Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $18 Avg. price ~ $29 18 tickets remaining! Tickets\r\n* Oct 03 2024 8:15 PM Thu Atlanta Falcons vs. Tampa Bay Buccaneers Mercedes-Benz Stadium | Capacity: 83000 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Prices from $42 Avg. price ~ $160 Available Tickets: 2033 Tickets\r\n* Oct 03 2024 8:30 PM Thu Fozzy Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Prices from $85 Avg. price ~ $124 10 tickets remaining! Tickets\r\n* Oct 04 2024 6:00 PM Fri Demon Hunter Heaven Stage at Masquerade - GA | Capacity: 1000 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $42 Avg. price ~ $73 Available Tickets: 55 Tickets\r\n* Oct 04 2024 7:00 PM Fri Kennesaw State Owls vs. Jacksonville State Gamecocks Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $22 Avg. price ~ $78 50 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Rehab Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Prices from $92 Avg. price ~ $107 2 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Sprints Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $36 Avg. price ~ $41 22 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Arrested Development The Ville 30214, 301 Lafayette Ave., Fayetteville, GA, US Prices from $59 Avg. price ~ $69 12 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Jon Pardi Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $21 Avg. price ~ $107 Available Tickets: 1013 Tickets\r\n* Oct 04 2024 7:30 PM Fri Conan Gray Cadence Bank Amphitheatre at Chastain Park | Capacity: 6000 30327, 4469 Stella Drive Northwest, Atlanta, GA, US Prices from $68 Avg. price ~ $135 Available Tickets: 131 Tickets\r\n* Oct 04 2024 7:30 PM Fri Atlanta Symphony Orchestra: How To Train Your Dragon In Concert Atlanta Symphony Hall | Capacity: 1762 30309, 1280 Peachtree St, Atlanta, GA, US Prices from $129 Avg. price ~ $225 10 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri The Hold Steady Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $61 Avg. price ~ $87 20 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri Anna Akana City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $47 Avg. price ~ $85 26 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri Kumail Nanjiani The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $45 Avg. price ~ $95 Available Tickets: 324 Tickets\r\n* Oct 04 2024 8:00 PM Fri Midland - Band Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $36 Avg. price ~ $73 Available Tickets: 67 Tickets\r\n* Oct 04 2024 8:00 PM Fri Sarah Kinsley The Loft - GA 30309, 1374 W. Peachtree Street, Atlanta, GA, US Prices from $35 Avg. price ~ $40 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri Friday Pilots Club Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $26 Avg. price ~ $59 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri MJ Lenderman and The Wind Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $39 Avg. price ~ $52 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri Marco Antonio Solis State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $120 Avg. price ~ $331 Available Tickets: 777 Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[The Slaps Upcoming Events: 2 Altar at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-slaps/)[Sepultura Upcoming Events: 2 Buckhead Theatre Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/sepultura/)[La Luz Upcoming Events: 2 Purgatory Stage at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/la-luz/)[Charli XCX Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/charli-xcx/)[Troye Sivan Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/troye-sivan/)[The Lion King Upcoming Events: 44 Fabulous Fox Theatre - Atlanta Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-lion-king/)[The Marley Brothers Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-marley-brothers/)[Stephen Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/stephen-marley/)[Ziggy Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/ziggy-marley/)[Will Burkart Upcoming Events: 2 City Winery - Atlanta Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/will-burkart/)[MJ Lenderman Upcoming Events: 4 Terminal West At King Plow Arts Center Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/mj-lenderman/)[Johnny Marr Upcoming Events: 2 The Eastern - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/johnny-marr/)[Elle King Upcoming Events: 2 The Tabernacle - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/elle-king/)[Horse Jumper of Love Upcoming Events: 2 Aisle 5 Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/horse-jumper-of-love/)[vaultboy Upcoming Events: 2 Vinyl - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/vaultboy/)[Atlanta Falcons Upcoming Events: 6 Mercedes-Benz Stadium Oct 3, 2024 8:15 PM](https://atlanta-ga.events/tickets/atlanta-falcons/)[Tampa Bay Buccaneers Upcoming Events: 3 Mercedes-Benz Stadium Oct 3, 2024 8:15 PM](https://atlanta-ga.events/tickets/tampa-bay-buccaneers/)[Fozzy Upcoming Events: 2 Madlife Stage \u0026 Studios Oct 3, 2024 8:30 PM](https://atlanta-ga.events/tickets/fozzy/)[Demon Hunter Upcoming Events: 2 Heaven Stage at Masquerade - GA Oct 4, 2024 6:00 PM](https://atlanta-ga.events/tickets/demon-hunter/)[Jacksonville State Gamecocks Football Upcoming Events: 3 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks-football/)[Rehab Upcoming Events: 2 Madlife Stage \u0026 Studios Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/rehab/)[Arrested Development Upcoming Events: 2 The Ville Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/arrested-development/)[Kennesaw State Owls Upcoming Events: 6 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls/)[Jacksonville State Gamecocks Upcoming Events: 3 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks/)[Jon Pardi Upcoming Events: 2 Ameris Bank Amphitheatre Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jon-pardi/)[Sprints Upcoming Events: 2 Altar at Masquerade - GA Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/sprints/)[Kennesaw State Owls Football Upcoming Events: 6 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls-football/)[Anna Akana Upcoming Events: 2 City Winery - Atlanta Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/anna-akana/)[The Hold Steady Upcoming Events: 4 Variety Playhouse Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-hold-steady/)[Kumail Nanjiani Upcoming Events: 2 The Tabernacle - GA Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/kumail-nanjiani/)[Atlanta Symphony Orchestra Upcoming Events: 18 Atlanta Symphony Hall Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[How to Train Your Dragon In Concert Upcoming Events: 4 Atlanta Symphony Hall Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/how-to-train-your-dragon-in-concert/)[Conan Gray Upcoming Events: 2 Cadence Bank Amphitheatre at Chastain Park Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/conan-gray/)[Friday Pilots Club Upcoming Events: 2 Vinyl - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/friday-pilots-club/)[Marco Antonio Solis Upcoming Events: 2 State Farm Arena - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/marco-antonio-solis/)[Sarah Kinsley Upcoming Events: 2 The Loft - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/sarah-kinsley/)[Midland - Band Upcoming Events: 2 Coca-Cola Roxy Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/midland-band/)[Gimme Gimme Disco Upcoming Events: 2 The Recher Theatre Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/gimme-gimme-disco/)[Zeke Beats Upcoming Events: 2 Aisle 5 Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/zeke-beats/)[Cozy Worldwide - 90s/2000s Hip Hop and R\u0026B Party Upcoming Events: 2 Ace of Spades Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/cozy-worldwide-90s-2000s-hip-hop-and-rb-party/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html"}
```
and got this result:
```
"Atlanta Events - October 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; October Events\r\n\r\n# Atlanta Events - October 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Concerts in Atlanta in October \u0026ndash; Atlanta\u0026rsquo;s biggest annual festivals \u0026ndash; Atlanta weather in October \u0026ndash; Things to do in Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From October 1 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; See September events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center with tournament prizes across 20 gaming contests. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; OYE Fest, October 6 2024Annual festival in Historic Fourth Ward Park celebrating Latinx art and music. More details\r\n\r\n\u0026ndash; La Cotorrisa Tour, October 6 2024Spanish-language comedy podcast show at the Buckhead Theatre. More details\r\n\r\n## Events From October 7 To Columbus Day Weekend, October 12-14 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Atlanta miniFEST, October 9 2024Community music and arts festival at Believe Music Hall. More details\r\n\r\n\u0026ndash; Taste Brookhaven, October 10 2024Food-tasting event in the gardens of the Hyatt Regency Atlanta Perimeter, featuring Brookhaven chefs and restaurants plus drinks and live entertainment, to benefit Children\u0026rsquo;s Healthcare of Atlanta. More details\r\n\r\n\u0026ndash; Jake Shane, October 10 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Scott Antique Market, October 10-13 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; Atsuko Okatsuka, October 11 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Pride Kickoff, October 11 2024After-hours LGBTQ\u002B party at the Georgia Aquarium. More details\r\n\r\n\u0026ndash; Wire \u0026amp; Wood Alpharetta, October 11-12 2024Songwriter music festival with performances across downtown Alpharetta. More details\r\n\r\n\u0026ndash; Joyce Meyer Live, October 11-12 2024Faith event at the Gas South Arena. More details\r\n\r\n\u0026ndash; Take Your Royal Position Conference, October 11-12 2024Faith and empowerment event in downtown Atlanta. More details\r\n\r\n\u0026ndash; Duluth Fall Home Expo, October 11-13 2024Home renovations trade show at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; Oakhurst Porchfest, October 12 2024Annual community music festival in Decatur over 100 live performances on Oakhurst neighborhood porches. More details\r\n\r\n\u0026ndash; Madison Avenue Derby, October 12 2024Annual races in Decatur in homemade, human-powered soapbox cars, to benefit local children\u0026rsquo;s charities. More details\r\n\r\n\u0026ndash; Politics War Room with James Carville \u0026amp; Al Hunt, October 12 2024Talk event at Variety Playhouse. More details\r\n\r\n\u0026ndash; Atlanta Pride Festival, October 12-13 2024One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more. More details\r\n\r\n\u0026ndash; Taste \u0026amp; Brews Fall Festival, October 12-13 2024Annual food and beer festival at Etowah River Park, Canton, featuring southern food and beers, musicians from across Georgia, arts and crafts, farmers market, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Old Fourth Ward Fall Arts Festival, October 12-13 2024Twice-annual arts festival in Historic Fourth Ward Park featuring local and Georgian artists, demonstrations, live entertainment, art activities, food and more. More details\r\n\r\n\u0026ndash; Chalktoberfest, October 12-13 2024Popular annual art and beer festival in Marietta Square, featuring professional chalk artists, 120\u002B craft beers, live music, arts and crafts, food trucks, amateur chalk art competition, and more. More details\r\n\r\n\u0026ndash; FashionFest, October 12-13 2024Festival of urban-popular fashion and streetwear at Ambient Plus Studio, with fashion and footwear vendors, food, music and more. More details\r\n\r\n\u0026ndash; Gold Over America Tour, October 13 2024Gymnastics performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Roswell Town Square Dinner Series, October 13 2024Outdoor community dining event at Roswell Town Square with food from The Mill Kitchen \u0026amp; Bar. More details\r\n\r\n\u0026ndash; Pride Parade Viewing Party, October 13 2024Rooftop LGBTQ\u002B party at the Fox Theatre, to coincide with the annual Pride parade. More details\r\n\r\n\u0026ndash; Joe Wong, October 14 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n## Events From October 15 To The Weekend Of October 19-20 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; David Cross \u0026ndash; End of the Beginning of the End, October 16 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jessica Kirson, October 18 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Sharon McMahon, October 18 2024Author talk at Center Stage. More details\r\n\r\n\u0026ndash; Stone Mountain Highland Games and Scottish Festival, October 18-20 2024Annual festival of Scottish heritage at Stone Mountain Park, with traditional Scottish athletics, music, dancing, food and drink, and vendors. All are welcome. More details\r\n\r\n\u0026ndash; Dracula: Presented by Gwinnett Ballet Theatre, October 18-20 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Georgia Metropolitan Dance Theatre presents Dracula, October 18-20 2024Ballet performance at the KSU Dance Theatre, Marietta Campus. More details\r\n\r\n\u0026ndash; A Taste of Chamblee, October 19 2024Annual food festival in downtown Chamblee, with food from neighborhood restaurants, craft vendors, children\u0026rsquo;s activities, football screenings, live performances, wine tent, beer garden and more. More details\r\n\r\n\u0026ndash; HarvestFest and Scarecrows In The Square, October 19 2024Annual fall festival in Marietta Square, with arts and crafts show, touch-a-truck, scarecrow displays, costume and pie eating contests, and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Boo! Halloween, October 19 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; Cherokee Heights Arts Festival, October 19 2024Free annual arts festival in the Cherokee Heights neighborhood, with local artists and makers, local musicians, art activities, food, beer and more. More details\r\n\r\n\u0026ndash; NBAF Celebration Under The Stars, October 19 2024Fundraising event for the National Black Arts Festival at Wolf Creek Amphitheater, with live performances and food and drink. More details\r\n\r\n\u0026ndash; Rickey Smiley, October 19 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Carlos Ballarta: Tlatoani, October 19 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Witch Perfect, October 19 2024Drag performance at Center Stage. More details\r\n\r\n\u0026ndash; L5P Halloween Festival and Parade, October 19-20 2024Popular annual Halloween celebration in Little Five Points, with live music, parade, haunted house, ghost tours and more. More details\r\n\r\n\u0026ndash; PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. More details\r\n\r\n\u0026ndash; Johns Creek Arts Festival, October 19-20 2024Annual arts and crafts festival in Johns Creek featuring 140 artisans from across the country. More details\r\n\r\n\u0026ndash; Abnormal Market and Uncommon Bazaar, October 19-20 2024Art market in association with the Old Fourth Ward Park Arts Festival. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Landlocked 8, October 20 2024Oyster and BBQ festival at Legacy Park, Decatur. More details\r\n\r\n\u0026ndash; MrBallen: The Strange, Dark, \u0026amp; Mysterious Live Tour, October 20 2024Podcast live show at the Coca-Cola Roxy. More details\r\n\r\n\u0026ndash; National Ballet of Ukraine, October 20 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Shakespeare Out of a Hat: Macbeth, October 20 2024Part-improvised play at the Shakespeare Tavern Playhouse. More details\r\n\r\n## Events From October 21 To The Weekend Of October 26-27 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Malcolm Gladwell in conversation with Virginia Prescott \u0026ndash; Revenge of the Tipping Point, October 22 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Jim Henson\u0026rsquo;s Labyrinth: In Concert, October 23 2024Film screening with live musical accompaniment at The Eastern. More details\r\n\r\n\u0026ndash; A Conversation with Larry David, October 24 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; UpTempo \u0026ndash; A Composer\u0026rsquo;s Voice: From the Serene to the Terrifying, October 24 2024Pre-concert event at Woodruff Arts Center aimed at teens and their families. More details\r\n\r\n\u0026ndash; Whitney Cummings: Big Baby, October 25 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Midtown Halloween Block Party, October 25 2024Halloween-themed block party and costume contest at Politan Row at Colony Square. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part I, October 25 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Benteen Park Fall Fest, October 26 2024Annual fall festival in Benteen Park, with live music, artist market and food vendors. More details\r\n\r\n\u0026ndash; Yappy Hour, October 26 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; Jerry Seinfeld, October 26 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Felipe Esparza, October 26 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Halloween in the Highland, October 26 2024Halloween-themed block party and costume contest in the Virginia-Highland neighborhood. More details\r\n\r\n\u0026ndash; Buckhead Creepy Crawl, October 26 2024Halloween-themed bar crawl and costume contest in the Buckhead entertainment district. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Air Dot Show Tour, October 26-27 2024Air show at Atlanta Regional Airport, Peachtree City. More details\r\n\r\n\u0026ndash; ONE Musicfest, October 26-27 2024Big annual festival of urban progressive music, in Piedmont Park. More details\r\n\r\n\u0026ndash; Menopause The Musical 2: Cruising Through \u0026lsquo;The Change\u0026rsquo;, October 26-27 2024Musical at the Gas South Theater. More details\r\n\r\n\u0026ndash; CompassionCon, October 27 2024Annual community wellness festival in Legacy Park, Decatur, with art, music, wellness activities, health services, food trucks and more. More details\r\n\r\n\u0026ndash; Hillary Clinton Live, October 27 2024Author talk at the Fox Theatre. More details\r\n\r\n## Events From October 28 To THe Weekend Of November 2-3 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Historical Marker Dedication: Booker T Washington High School, October 29 2024Historical marker dedication by the Georgia Historical Society at Atlanta\u0026rsquo;s first public secondary school for Black students, at Booker T Washington High School, 45 Whitehouse Drive SW. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part II, October 31 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Native American Festival and Pow Wow, October 31 - November 3 2024Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details\r\n\r\n\u0026ndash; Two Hot Takes Podcast: No Takes Left Behind, November 1 2024Podcast live show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Black Girls Rock!, November 1 2024Annual awards ceremony at the Fox Theatre celebrating the accomplishments of Black women. More details\r\n\r\n\u0026ndash; Ghostbusters in Concert, November 1-2 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Party With Impact \u0026ndash; Positive Impact Health Centers, November 1-2 2024Comedy show at the Gas South Theater to benefit LGBTQ\u002B healthcare services in Atlanta. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, November 1-3 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Rescue Dog Games: Howl-o-Ween Festival, November 2 2024Annual pet-friendly festival in Piedmont Park, with dog activities, dog costume parade and contest, pet-centric vendors, live music, food and beer, adoptable pets, and more. More details\r\n\r\n\u0026ndash; Chomp and Stomp, November 2 2024Annual neighborhood festival in Cabbagetown, with chili, beer, bluegrass music and more, to benefit the Friends of Cabbagetown Conservancy. More details\r\n\r\n\u0026ndash; D\u0026iacute;a de los Muertos Festival, November 2 2024Free family festival at Woodruff Arts Center, with live music, mariachi band, vendors, food trucks, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Suwanee Wine Fest, November 2 2024Annual festival in Suwanee Town Center Park, with wine samples from across the world, Georgia wines, live music, arts and crafts vendors, food and more. More details\r\n\r\n\u0026ndash; \u0026ldquo;Honoring Our Past Through Stories\u0026rdquo; Historic Landscape Preservation Grant Fund Benefit, November 2 2024Outdoor event at the Marietta Educational Garden Center to benefit the Garden Club of Georgia\u0026rsquo;s Historic Landscape Preservation Grant, with speakers, plein air painters, exhibits, music and more. More details\r\n\r\n\u0026ndash; Chastain Park Arts Festival, November 2-3 2024Popular twice-annual arts festival at Chastain Park, with 175\u002B arts and crafts vendors, food trucks, local musicians, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; The Elegant Elf Marketplace, November 2-3 2024Annual holiday marketplace with 80\u002B vendors at the Sandy Springs Performing Arts \u0026amp; Conference Center, to benefit The Sandy Springs Society. More details\r\n\r\n\u0026ndash; Jeff Arcuri: The Full Beans Fall Tour, November 3 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Disney Jr Live On Tour: Let\u0026rsquo;s Play, November 3 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026rarr; See November events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024"}
```
and got this result:
```
"Atlanta Concerts, Festivals, Tickets \u0026amp; Tour Dates 2024 \u0026amp; 2025 \u2013 Songkick* Atlanta concerts Atlanta concerts Atlanta concerts Michel\u00B4le The Masquerade - Purgatory Marc Broussard The Eastern Two Friends Coca-Cola Roxy Elsa y Elmar The Masquerade - Heaven Perpetual Groove Variety Playhouse Hotline TNT Eyedrum Gallery Will Sparks District Atlanta Madison Ryann Ward The Loft Pom Pom Squad The Masquerade - Purgatory The Secret Sisters Capitol Theatre See all Atlanta concerts (Change\u0026nbsp;location) Today \u00B7 Next 7 days \u00B7 Next 30 days Artists Popular artists in Atlanta Trending artists worldwide The Verve Catfish and the Bottlemen The Wombats Hardwell George Ezra Midland KATSEYE Los Farmerz Kapo dublon Get your tour dates seen by one billion fans: Sign up as an artist Festivals Tourbox for artists\r\n* Search for events or artists Sign up Log in\r\n\r\nSorry, you have to enable javascript to view this page![](/)* \r\n* Get the app\r\n* Home\r\n* Atlanta concerts\r\n* Change\u0026nbsp;location\r\n* Popular Artists\r\n* Festivals\r\n* Tourbox for artists\r\n* Live streams\r\n* Language English Fran\u00E7ais Espa\u00F1ol\r\n* Deutsch Portugu\u00EAs\r\n* Log in to your account\r\n* Sign up\r\n\r\n* Find concerts Sorry, you have to enable javascript to view this page!\r\n* Concerts near you\r\n* Your artists\r\n* Popular artists\r\n* Tourbox for artists\r\n\r\n# Concerts in Atlanta\r\n\r\nFind tickets to all live music, concerts, tour dates and festivals in and around Atlanta.\r\n\r\nCurrently there are 1366 upcoming events.\r\n\r\nFilter by artist* All\r\n* Your favorite artists\r\n\r\nFilter by date* All\r\n* Tonight\r\n* This weekend\r\n* This month\r\n* Nov\r\n* Dec\r\n* Jan\r\n* 2024\r\n* 2025\r\n* \r\n\r\nFilter by genre* All\r\n* Rock\r\n* Comedy\r\n* Pop\r\n* Hip-Hop\r\n* R\u0026amp;B\r\n* Indie \u0026amp; Alt\r\n* Electronic\r\n* Country\r\n* Classical\r\n* Metal\r\n* Latin\r\n* Folk \u0026amp; Blues\r\n* Jazz\r\n* Funk \u0026amp; Soul\r\n* Reggae\r\n* \r\n\r\n## 906 upcoming concerts\r\n\r\n* Wednesday 02 October 2024\r\n* The Hives and Bad Nerves Tabernacle, Atlanta, GA, US\r\n* The Beaches and Charlie Houston The Masquerade - Heaven, Atlanta, GA, US\r\n* Jinjer, HANABIE, and Born of Osiris Buckhead Theatre, Atlanta, GA, US\r\n* Dayseeker Alpha Wolf, Kingdom of Giants, and Catch Your Breath The Eastern, Atlanta, GA, US\r\n* Lola Young and Bug Eyed Center Stage Theater, Atlanta, GA, US\r\n* Zheani The Buttress and ZAND Terminal West, Atlanta, GA, US\r\n* Moss Georgia Theatre, Athens, GA, US\r\n* The Moss(US) hey, nothing Georgia Theater, Athens, GA, US\r\n* T Bone Burnett Variety Playhouse, Atlanta, GA, US\r\n* QUIVERS The Inner Space, Atlanta, GA, US\r\n* See more concerts on this date\r\n* Thursday 03 October 2024\r\n* Charli XCX and Troye Sivan State Farm Arena, Atlanta, GA, US\r\n* Ziggy Marley and Marley Brothers Stephen Marley, Julian Marley, Ky-mani Marley, and Damian \u0026#39;Jr Gong\u0026#39; Marley Cellairis Amphitheatre at Lakewood, Atlanta, GA, US\r\n* Sepultura, Obituary, Agnostic Front, and Claustrophobia Buckhead Theatre, Atlanta, GA, US\r\n* Elle King Tabernacle, Atlanta, GA, US\r\n* La Luz and Mia Joy The Masquerade - Purgatory, Atlanta, GA, US\r\n* Johnny Marr The Eastern, Atlanta, GA, US\r\n* The Hold Steady The Earl, East Atlanta, GA, US\r\n* Horse Jumper of Love Truth Club and Joyer Aisle 5, Atlanta, GA, US\r\n* Noah Gundersen Eddie\u0026#39;s Attic, Decatur, GA, US\r\n* MJ Lenderman Terminal West, Atlanta, GA, US\r\n* See more concerts on this date\r\n* Friday 04 October 2024\r\n* Conan Gray Maisie Peters Cadence Bank Amphitheatre at Chastain Park, Atlanta, GA, US\r\n* Jon Pardi Priscilla Block and Larry Fleet Ameris Bank Amphitheatre, Alpharetta, GA, US\r\n* Marco Antonio Solis State Farm Arena, Atlanta, GA, US\r\n* The Hold Steady Kevin Morby Variety Playhouse, Atlanta, GA, US\r\n* Friday Pilots Club, OSTON, and Maggie Miles Vinyl at Center Stage, Atlanta, GA, US\r\n* Sarah Kinsley and Ha Vay The Loft, Atlanta, GA, US\r\n* Demon Hunter Impending Doom, War of Ages, and Opponent The Masquerade, Atlanta, GA, US\r\n* The War and Treaty Eddie\u0026#39;s Attic, Decatur, GA, US\r\n* SPRINTS and Slow Fiction The Masquerade - Altar, Atlanta, GA, US\r\n* Saturday 05 October 2024\r\n* Jason Aldean, Hailey Whitters, Chase Matthew, Austin Snell, and Dee Jay Silver Atrium Health Amphitheater, Macon, GA, US\r\n* Nothing But Thieves Coca-Cola Roxy, Atlanta, GA, US\r\n* Tori Kelly and Maisy Kay Tabernacle, Atlanta, GA, US\r\n* The Hold Steady Kevin Morby Variety Playhouse, Atlanta, GA, US\r\n* Cody Jinks Ameris Bank Amphitheatre, Alpharetta, GA, US\r\n* Rapsody, NIKO BRIM, and Lara The Masquerade - Hell, Atlanta, GA, US\r\n* Trombone Shorty \u0026amp; Orleans Avenue Big Freedia The Eastern, Atlanta, GA, US\r\n* Jesus Adrian Romero Cobb Energy Performing Arts Centre, Atlanta, GA, US\r\n* Carrtoons Vinyl at Center Stage, Atlanta, GA, US\r\n* NOYSE (IT) District Atlanta, Atlanta, GA, US\r\n* See more concerts on this date\r\n* Sunday 06 October 2024\r\n* Violent Femmes and Atlanta Symphony Orchestra Atlanta Symphony Hall, Atlanta, GA, US\r\n* Kirk Franklin, Yolanda Adams, Marvin Sapp, The Clark Sisters, and Fred Hammond State Farm Arena, Atlanta, GA, US\r\n* Kidz Bop Kids Ameris Bank Amphitheater, Alpharetta, GA, US\r\n* nobigdyl., Jon Keith, and aklesso Vinyl at Center Stage, Atlanta, GA, US\r\n* David Bazan Undertow Show, Atlanta, GA, US\r\n* Heavy Heavy Terminal West, Atlanta, GA, US\r\n* Landon Conrath Kevian Kraemer The Masquerade - Purgatory, Atlanta, GA, US\r\n* David Bazan Undertow Show, Athens, GA, US\r\n* Sunday 06 October 2024 \u2013 Sunday 06 October 2024\r\n* OYE FEST 2024 La India, India, akapellah, Farina, Reyna Tropical, and LATENIGHTJIGGY Historic Fourth Ward Park, Atlanta, GA, US\r\n* Monday 07 October 2024\r\n* Remi Wolf The Eastern, Atlanta, GA, US\r\n* Charly Bliss and Pronoun The Masquerade - Hell, Atlanta, GA, US\r\n\r\n\u0026#8592; Previous*1*[2](/en/metro-areas/4120-us-atlanta/october-2024?page=2#metro-area-calendar)[3](/en/metro-areas/4120-us-atlanta/october-2024?page=3#metro-area-calendar)[4](/en/metro-areas/4120-us-atlanta/october-2024?page=4#metro-area-calendar)[5](/en/metro-areas/4120-us-atlanta/october-2024?page=5#metro-area-calendar)[6](/en/metro-areas/4120-us-atlanta/october-2024?page=6#metro-area-calendar)[7](/en/metro-areas/4120-us-atlanta/october-2024?page=7#metro-area-calendar)...[18](/en/metro-areas/4120-us-atlanta/october-2024?page=18#metro-area-calendar)[19](/en/metro-areas/4120-us-atlanta/october-2024?page=19#metro-area-calendar)[Next \u0026#8594;](/en/metro-areas/4120-us-atlanta/october-2024?page=2#metro-area-calendar)## Popular tickets in Atlanta\r\n\r\n[](/concerts/42191466-michelle-at-masquerade-purgatory)[Michel\u00B4le](/artists/10004349-michelle)[Wed 12 Feb 2025 The Masquerade - Purgatory Atlanta, GA, US](/concerts/42191466-michelle-at-masquerade-purgatory)[](/concerts/42188342-marc-broussard-at-eastern)[Marc Broussard](/artists/135964-marc-broussard)[Thu 20 Feb 2025 The Eastern Atlanta, GA, US](/concerts/42188342-marc-broussard-at-eastern)[](/concerts/42178902-two-friends-at-cocacola-roxy)[Two Friends](/artists/8423523-two-friends)[Sun 16 Feb 2025 Coca-Cola Roxy Atlanta, GA, US](/concerts/42178902-two-friends-at-cocacola-roxy)[](/concerts/42178555-elsa-y-elmar-at-masquerade-heaven)[Elsa y Elmar](/artists/7360644-elsa-y-elmar)[Fri 31 Jan 2025 The Masquerade - Heaven Atlanta, GA, US](/concerts/42178555-elsa-y-elmar-at-masquerade-heaven)[](/concerts/42177535-perpetual-groove-at-variety-playhouse)[Perpetual Groove](/artists/375801-perpetual-groove)[Sat 15 Mar 2025 Variety Playhouse Atlanta, GA, US](/concerts/42177535-perpetual-groove-at-variety-playhouse)[](/concerts/42168819-hotline-tnt-at-eyedrum-gallery)[Hotline TNT](/artists/9615249-hotline-tnt)[Fri 22 Nov 2024 Eyedrum Gallery Atlanta, GA, US](/concerts/42168819-hotline-tnt-at-eyedrum-gallery)[](/concerts/42189160-will-sparks-at-district-atlanta)[Will Sparks](/artists/6286599-will-sparks)[Fri 06 Dec 2024 District Atlanta Atlanta, GA, US](/concerts/42189160-will-sparks-at-district-atlanta)[](/concerts/42181385-madison-ryann-ward-at-loft)[Madison Ryann Ward](/artists/9926024-madison-ryann-ward)[Fri 25 Oct 2024 The Loft Atlanta, GA, US](/concerts/42181385-madison-ryann-ward-at-loft)[](/concerts/42178512-pom-pom-squad-at-masquerade-purgatory)[Pom Pom Squad](/artists/9047119-pom-pom-squad)[Sat 29 Mar 2025 The Masquerade - Purgatory Atlanta, GA, US](/concerts/42178512-pom-pom-squad-at-masquerade-purgatory)[](/concerts/42165032-secret-sisters-at-capitol-theatre)[The Secret Sisters](/artists/3470011-secret-sisters)[Sat 14 Dec 2024 Capitol Theatre Macon, GA, US](/concerts/42165032-secret-sisters-at-capitol-theatre)[](/concerts/42189974-jimmy-brown-at-vinyl-at-center-stage)[Jimmy Brown](/artists/419483-jimmy-brown)[Wed 04 Dec 2024 Vinyl at Center Stage Atlanta, GA, US](/concerts/42189974-jimmy-brown-at-vinyl-at-center-stage)[](/concerts/42190922-trauma-ray-at-masquerade-altar)[Trauma Ray](/artists/9601364-trauma-ray)[Tue 19 Nov 2024 The Masquerade - Altar Atlanta, GA, US](/concerts/42190922-trauma-ray-at-masquerade-altar)[](/concerts/42188370-donny-benet-at-georgia-theatre)[Donny Benet](/artists/4330741-donny-benet)[Thu 23 Jan 2025 Georgia Theatre Athens, GA, US](/concerts/42188370-donny-benet-at-georgia-theatre)[](/concerts/42161561-preservation-hall-jazz-band-at-variety-playhouse)[Preservation Hall Jazz Band](/artists/54186-preservation-hall-jazz-band)[Thu 05 Dec 2024 Variety Playhouse Atlanta, GA, US](/concerts/42161561-preservation-hall-jazz-band-at-variety-playhouse)[](/concerts/42175815-zach-top-at-buckhead-theatre)[Zach Top](/artists/10244239-zach-top)[Sat 08 Mar 2025 Buckhead Theatre Atlanta, GA, US](/concerts/42175815-zach-top-at-buckhead-theatre)[](/concerts/42196029-callous-daoboys-at-masquerade-altar)[Callous Daoboys](/artists/8917964-callous-daoboys)[Sat 07 Dec 2024 The Masquerade - Altar Atlanta, GA, US](/concerts/42196029-callous-daoboys-at-masquerade-altar)[](/concerts/42200446-whigs-at-40-watt-club)[The Whigs](/artists/83496-whigs)[Fri 17 Jan 2025 40 Watt Club Athens, GA, US](/concerts/42200446-whigs-at-40-watt-club)[](/concerts/42171869-orions-belte-at-earl)[Orions Belte](/artists/9492184-orions-belte)[Tue 18 Mar 2025 The Earl East Atlanta, GA, US](/concerts/42171869-orions-belte-at-earl)[](/concerts/42155910-blackberry-smoke-at-macon-city-auditorium)[Blackberry Smoke](/artists/568895-blackberry-smoke)[Tue 31 Dec 2024 Macon City Auditorium Macon, GA, US](/concerts/42155910-blackberry-smoke-at-macon-city-auditorium)[](/concerts/42184839-hulvey-at-tabernacle)[Hulvey](/artists/10273038-hulvey)[Thu 06 Mar 2025 Tabernacle Atlanta, GA, US](/concerts/42184839-hulvey-at-tabernacle)[](/concerts/42185356-pigeon-pit-at-wallers)[pigeon pit](/artists/8609119-pigeon-pit)[Mon 28 Oct 2024 Wallers Decatur, GA, US](/concerts/42185356-pigeon-pit-at-wallers)[](/concerts/42161324-daily-bread-at-eastern)[Daily Bread](/artists/1569020-daily-bread)[Mon 30 Dec 2024 The Eastern Atlanta, GA, US](/concerts/42161324-daily-bread-at-eastern)[](/concerts/42162020-dhruv-at-terminal-west)[Dhruv](/artists/2565406-dhruv)[Tue 04 Feb 2025 Terminal West Atlanta, GA, US](/concerts/42162020-dhruv-at-terminal-west)[](/concerts/42167656-hugel-at-district-atlanta)[HUGEL](/artists/8589694-hugel)[Tue 31 Dec 2024 District Atlanta Atlanta, GA, US](/concerts/42167656-hugel-at-district-atlanta)[](/concerts/42175834-lauren-sanderson-at-masquerade-hell)[Lauren Sanderson](/artists/8648744-lauren-sanderson)[Wed 05 Feb 2025 The Masquerade - Hell Atlanta, GA, US](/concerts/42175834-lauren-sanderson-at-masquerade-hell)[](/concerts/42183103-sunny-sweeney-at-boot-barn-hall-at-bourbon-brothers)[Sunny Sweeney](/artists/536209-sunny-sweeney)[Sat 19 Oct 2024 Boot Barn Hall at Bourbon Brothers Gainesville, GA, US](/concerts/42183103-sunny-sweeney-at-boot-barn-hall-at-bourbon-brothers)[](/concerts/42164519-big-head-todd-and-the-monsters-at-variety-playhouse)[Big Head Todd \u0026amp; The Monsters](/artists/506841-big-head-todd-and-the-monsters)[Fri 14 Mar 2025 Variety Playhouse Atlanta, GA, US](/concerts/42164519-big-head-todd-and-the-monsters-at-variety-playhouse)[](/concerts/42178745-rare-americans-at-masquerade-hell)[Rare Americans](/artists/9861214-rare-americans)[Sat 15 Mar 2025 The Masquerade - Hell Atlanta, GA, US](/concerts/42178745-rare-americans-at-masquerade-hell)[](/concerts/42183716-yhapojj-at-masquerade-hell)[Yhapojj](/artists/10275404-yhapojj)[Sun 20 Oct 2024 The Masquerade - Hell Atlanta, GA, US](/concerts/42183716-yhapojj-at-masquerade-hell)[](/concerts/42165670-maria-bamford-at-buckhead-theatre)[Maria Bamford](/artists/73811-maria-bamford)[Sat 25 Jan 2025 Buckhead Theatre Atlanta, GA, US](/concerts/42165670-maria-bamford-at-buckhead-theatre)[](/concerts/42174544-tim-heidecker-at-terminal-west)[Tim Heidecker](/artists/2319898-tim-heidecker)[Mon 17 Feb 2025 Terminal West Atlanta, GA, US](/concerts/42174544-tim-heidecker-at-terminal-west)[](/concerts/42167960-cool-company-at-masquerade-purgatory)[Cool Company](/artists/7031174-cool-company)[Fri 22 Nov 2024 The Masquerade - Purgatory Atlanta, GA, US](/concerts/42167960-cool-company-at-masquerade-purgatory)[](/concerts/42176334-kindred-the-family-soul-at-city-winery-atlanta)[Kindred the Family Soul](/artists/502439-kindred-the-family-soul)[Sat 22 Mar 2025 City Winery Atlanta Atlanta, GA, US](/concerts/42176334-kindred-the-family-soul-at-city-winery-atlanta)[](/concerts/42171767-coucou-chloe-at-masquerade-purgatory)[COUCOU CHLOE](/artists/9272354-coucou-chloe)[Wed 20 Nov 2024 The Masquerade - Purgatory Atlanta, GA, US](/concerts/42171767-coucou-chloe-at-masquerade-purgatory)[](/concerts/42176338-iris-dement-at-city-winery-atlanta)[Iris DeMent](/artists/364629-iris-dement)[Wed 29 Jan 2025 City Winery Atlanta Atlanta, GA, US](/concerts/42176338-iris-dement-at-city-winery-atlanta)[](/concerts/42171548-ryan-oakes-at-masquerade-purgatory)[Ryan Oakes](/artists/7996803-ryan-oakes)[Sun 01 Dec 2024 The Masquerade - Purgatory Atlanta, GA, US](/concerts/42171548-ryan-oakes-at-masquerade-purgatory)[](/concerts/42169671-boogie-t-at-believe-music-hall)[Boogie T](/artists/8740419-boogie-t)[Sat 14 Dec 2024 Believe Music Hall Atlanta, GA, US](/concerts/42169671-boogie-t-at-believe-music-hall)[](/concerts/42180824-enamour-at-district-atlanta)[Enamour](/artists/9000439-enamour)[Fri 15 Nov 2024 District Atlanta Atlanta, GA, US](/concerts/42180824-enamour-at-district-atlanta)[](/concerts/42165705-kaonashi-at-masquerade-altar)[Kaonashi](/artists/8683684-kaonashi)[Wed 20 Nov 2024 The Masquerade - Altar Atlanta, GA, US](/concerts/42165705-kaonashi-at-masquerade-altar)[](/concerts/42167961-dirty-nil-at-masquerade-altar)[The Dirty Nil](/artists/3468641-dirty-nil)[Wed 22 Jan 2025 The Masquerade - Altar Atlanta, GA, US](/concerts/42167961-dirty-nil-at-masquerade-altar)[](/concerts/42169412-binomio-de-oro-at-atlanta-eagles-arena)[Binomio de Oro](/artists/45810-binomio-de-oro)[Sat 16 Nov 2024 Atlanta Eagles Arena Atlanta, GA, US](/concerts/42169412-binomio-de-oro-at-atlanta-eagles-arena)[](/concerts/42169355-mishka-at-eddies-attic)[Mishka](/artists/390040-mishka)[Tue 08 Oct 2024 Eddie\u0026#39;s Attic Decatur, GA, US](/concerts/42169355-mishka-at-eddies-attic)[](/concerts/42185650-ouija-macc-at-masquerade-altar)[Ouija Macc](/artists/9787914-ouija-macc)[Thu 14 Nov 2024 The Masquerade - Altar Atlanta, GA, US](/concerts/42185650-ouija-macc-at-masquerade-altar)[](/concerts/42134458-rated-r-at-believe-music-hall)[Rated R](/artists/336214-rated-r)[Fri 06 Dec 2024 Believe Music Hall Atlanta, GA, US](/concerts/42134458-rated-r-at-believe-music-hall)[](/concerts/42193346-bayker-blankenship-at-eddies-attic)[Bayker Blankenship](/artists/10346823-bayker-blankenship)[Thu 10 Oct 2024 Eddie\u0026#39;s Attic Decatur, GA, US](/concerts/42193346-bayker-blankenship-at-eddies-attic)[](/concerts/42153477-debbii-dawson-at-tabernacle)[Debbii Dawson](/artists/10259557-debbii-dawson)[Fri 07 Mar 2025 Tabernacle Atlanta, GA, US](/concerts/42153477-debbii-dawson-at-tabernacle)[](/concerts/42162034-maddy-oneal-at-eastern)[Maddy O\u0026#39;Neal](/artists/8754164-maddy-oneal)[Mon 30 Dec 2024 The Eastern Atlanta, GA, US](/concerts/42162034-maddy-oneal-at-eastern)[](/concerts/42176342-jeff-bradshaw-at-city-winery-atlanta)[Jeff Bradshaw](/artists/426159-jeff-bradshaw)[Fri 11 Oct 2024 City Winery Atlanta Atlanta, GA, US](/concerts/42176342-jeff-bradshaw-at-city-winery-atlanta)[](/concerts/42176360-jazzmeia-horn-at-city-winery-atlanta)[Jazzmeia Horn](/artists/9273949-jazzmeia-horn)[Mon 11 Nov 2024 City Winery Atlanta Atlanta, GA, US](/concerts/42176360-jazzmeia-horn-at-city-winery-atlanta)[](/concerts/42169399-elucid-at-529)[Elucid](/artists/867757-elucid)[Tue 01 Apr 2025 529 Atlanta, GA, US](/concerts/42169399-elucid-at-529)[](/concerts/42164687-grandville-at-georgia-theatre)[Grandville](/artists/385285-grandville)[Wed 06 Nov 2024 Georgia Theatre Athens, GA, US](/concerts/42164687-grandville-at-georgia-theatre)[](/concerts/42175334-flea-circus-at-529)[Flea Circus](/artists/2460086-flea-circus)[Thu 10 Oct 2024 529 Atlanta, GA, US](/concerts/42175334-flea-circus-at-529)[](/concerts/42195391-hail-gail-at-529)[Hail Gail](/artists/10243905-hail-gail)[Sat 02 Nov 2024 529 Atlanta, GA, US](/concerts/42195391-hail-gail-at-529)[](/concerts/42197205-shamba-at-novel-west-midtown)[Shamba](/artists/10347737-shamba)[Thu 17 Oct 2024 Novel West Midtown Atlanta, GA, US](/concerts/42197205-shamba-at-novel-west-midtown)* Home\r\n* About us\r\n* Get the app\r\n* Blog\r\n* Jobs\r\n* Support\r\n* Most popular charts\r\n* Festivals\r\n* News\r\n\r\n* Tourbox for artists\r\n* Campaigns for promoters\r\n* API information\r\n* Brand guidelines\r\n* Community guidelines\r\n* Terms of use\r\n* Privacy policy\r\n* Cookies settings\r\n* Cookies policy\r\n* Security\r\n* Live streams\r\n\r\nLanguage\r\n\r\n[English](https://www.songkick.com/metro-areas/4120-us-atlanta)[Fran\u00E7ais](https://www.songkick.com/fr/metro-areas/4120-us-atlanta)[Espa\u00F1ol](https://www.songkick.com/es/metro-areas/4120-us-atlanta)[Deutsch](https://www.songkick.com/de/metro-areas/4120-us-atlanta)[Portugu\u00EAs](https://www.songkick.com/pt/metro-areas/4120-us-atlanta)Get your tour dates seen everywhere.\r\n\r\n[Sign up as an artist](//tourbox.songkick.com/?utm_medium=referral\u0026amp;utm_source=songkick.com\u0026amp;utm_campaign=visitor)Official merchandise partner[](https://www.emp.co.uk/band-merch/)* \u0026nbsp;Follow us\r\n* \u0026nbsp;Like us\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/"}
```
and got this result:
```
"Response status code does not indicate success: 403 (Forbidden)."
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year"}
```
and got this result:
```
"Things to Do in Atlanta in 2024: Events, festivals, and exhibits - Thrillist[Skip to main content](#main-content)[Thrillist](/)[Subscribe](/newsletters)[Go to Navigation](#main-navigation)### Navigation\r\n\r\n* CitiesAtlantaAustinBostonChicagoDallasDenverHoustonLas VegasLos AngelesMiamiNashvilleNew OrleansNew YorkPhiladelphiaPhoenixSan DiegoSan FranciscoWashington DC\r\n* Plan Your TripTips \u0026amp; HacksWhat to PackLodgingFlightsTrainsCruisesRoad TripsBudgetSplurge\r\n* Guides \u0026amp; DestinationsNeighborhood GuidesMapsTravel ItinerariesTravel InspirationWeekend GetawaysBig CitiesSmall TownsHidden GemsInternational TripsMountainsDesertsForestsBeaches\r\n* ExperiencesArts \u0026amp; CultureCannabisFamilyFood \u0026amp; DrinkHikingLGBTQ\u002BMusicNightlifeOutdoors \u0026amp; NatureShopping \u0026amp; StyleSportsStargazingWellness\r\n* News \u0026amp; TrendsNewsTrends\r\n* Video\r\n\r\n* Atlanta\r\n* Events\r\n* Cities\r\n\r\n# 23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming\r\n\r\n## Make plans to attend these annual events in Atlanta that keep the party going all year.\r\n\r\nBy[Okla Jones](/authors/okla-jones)Updated on July 9, 2024 at 12:40 PM![performance at One Musicfest in Atlanta](https://assets3.thrillist.com/v1/image/3181949/414x310/crop;webp=auto;jpeg_quality=60;progressive.jpg)\r\n![performance at One Musicfest in Atlanta](https://assets3.thrillist.com/v1/image/3181949/414x310/crop;webp=auto;jpeg_quality=60;progressive.jpg)\r\nPhoto courtesy of One MusicfestPhoto courtesy of One MusicfestKnown as the epicenter of hip-hop and center of rich history, Atlanta remains one of the greatest spots in America for a trip. Locals like me know, there\u2019s always something going down in the city. Throughout the year, Georgia\u2019s capital is buzzing with long standing festivals, iconic sporting events, and so much more.\r\n\r\nPeople gravitate towards Atlanta for many reasons, so whether you love the food, music, or culture in the 404, there\u2019s always something going on here to accommodate your needs. While some folks are more of the planning type, others may be more spontaneous, so check out a list of the best experiences that city has to offer this year to make plans.\r\n\r\n## January\r\n\r\n### MLK Day Weekend\r\n\r\nJanuary 17\u201320, 2025The King CenterThere\u2019s no better way to celebrate the birthday of Dr. Martin Luther King, Jr. than by visiting The King Center in Old Fourth Ward. In addition to featuring the Freedom Hall exhibition space, this space houses the beautiful marble crypt that serves as the final resting place for King and his wife, Coretta Scott King. Furthermore, MLK\u2019s birth home is also within walking distance of the King Center, and you can schedule a tour in advance here.Ready to go? Book here.\r\n\r\n## February\r\n\r\n![man holding a plate of barbecue at Beer, Bourbon \u0026amp; BBQ Festival atlanta](https://assets3.thrillist.com/v1/image/3181938/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![man holding a plate of barbecue at Beer, Bourbon \u0026amp; BBQ Festival atlanta](https://assets3.thrillist.com/v1/image/3181938/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of Beer, Bourbon \u0026amp; BBQ Festival### Beer, Bourbon \u0026amp; BBQ Festival\r\n\r\n2025 dates TBDAtlantic StationCelebrating its 13th year, the highly anticipated Beer, Bourbon \u0026amp; BBQ Festival will return to Atlantic Station early next year. With a selection of more than 60 beers, 40 bourbons, and lots of barbecue, this festival is sure to leave everyone in attendance both full and satisfied. In my opinion, the long liquor list is one of the best parts of this festival, so keep your souvenir glass handy when you\u2019re traveling from table to table so you can indulge. The onsite vendors offer items such as pulled pork, whole hogs, ribs, brisket, chickens, and sausages. You can also listen to live music on the main stage from a few of the best bands in the South.Ready to go? Book here.\r\n\r\n## March\r\n\r\n![atlanta st patricks day parade](https://assets3.thrillist.com/v1/image/3181939/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![atlanta st patricks day parade](https://assets3.thrillist.com/v1/image/3181939/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nSt. Patrick\u0026#x27;s Day parade in Atlanta|Photo by alans1948/Flickr### St. Patrick\u2019s Day Parade\r\n\r\nMarch 15, 2025MidtownYear after year, the Atlanta St. Patrick\u2019s Day Parade is a staple of the spring season and one of the city\u2019s oldest events. Running from Arts Center to Cypress Street through Midtown, along the way you\u2019ll see floats, dancers, and drum and bagpipe corps, all led by the grand marshals, who are usually Irish dignitaries or local figures. Don\u2019t forget, the 5K starts the day, so get there early for a nice run.Ready to go? This event is free.\r\n\r\n### Art, Beats \u002B Lyrics\r\n\r\n2025 dates TBDPullman YardsEvery year, Art, Beats \u002B Lyrics returns for an event in its original home of Atlanta. Presented by Jack Daniel\u0027s Tennessee Honey, this traveling urban art and music exhibition that supports visionary artists and musicians who exude traits of authenticity, creativity, and individuality. These uniquely curated showcases blur the line between artist and audience and have had performances from some of the biggest artists in the business.Ready to go? Book here.\r\n\r\n## April\r\n\r\n### Atlanta Film Festival\r\n\r\nApril 24\u2013May 4, 2025Multiple locationsThe Atlanta Film Festival is one of the largest celebrations of cinema in the entire South. Next year, attendees can enjoy thousands of films by creators all over the world. The ATLFF also focuses on projects directed by women, non-binary, BIPOC, and Georgia residents. For 11 days, the city of Atlanta will be the epicenter of the film industry. \u201CIt\u2019s one of those events that you have to attend if you respect the craft of filmmaking,\u201D says Van Steward, a local actor and director. \u201CCreativity at its best.\u201DReady to go? Book here.\r\n\r\n### Black Effect Podcast Festival\r\n\r\n2025 dates TBDPullman YardsI\u2019ve attended this event for the past two years, and I believe it\u2019s one of the best festivals that the city has to offer. The Black Effect Podcast Fest features music, delicious food, and more. There\u2019s also in-person renditions from popular shows for the company, along with informative panels, and a variety of other activities including immersive activations, a local vendor Marketplace and Networking Sessions with iHeartMedia and Black Effect staff.Ready to go? Book here.\r\n\r\n### David C. Driskell Prize Gala\r\n\r\n2025 dates TBDThe High Museum of ArtFor me, The High is one of the greatest establishments that the city has to offer. The art is exquisite, and the events are even better. Every year, the museum hosts one of the most anticipated galas in the country. Established in 2005, the Driskell Prize recognizes field-defining contributions to Black art by some of the leading scholars and artists from around the country, and brings about notable figures in the art world.Ready to go? Book here.\r\n\r\n## May\r\n\r\n![beer at a cinco de mayo party in atlanta](https://assets3.thrillist.com/v1/image/3181941/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![beer at a cinco de mayo party in atlanta](https://assets3.thrillist.com/v1/image/3181941/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of La Parrilla### Cinco De La Parrilla\r\n\r\nMay 5, 2025La ParrillaCinco De Mayo has celebrations popping up all over the city. The celebration at La Parrilla on Howell Mill stands above the rest. \u201CI enjoy and look forward to this event every year,\u201D says Georgia native Tangi Hoskins. During the day, attendees can enjoy live music, food and drink specials, and of course\u2014a great time.Ready to go? This event is free to attend.\r\n\r\n### Atlanta Caribbean Carnival\r\n\r\nMay 24, 2025Westside ParkNo matter where you are in the world, the Caribbean\u2019s influence can be felt in some shape or form. For almost 40 years, the Atlanta Caribbean Carnival has promoted a broader understanding and deeper appreciation of the Caribbean and will return for another event during Memorial Day weekend. Held in Westside Park, you and many others can enjoy music, dancing, food, children\u0027s activities, and much more while paying homage to a culture that has become so deeply ingrained in our country.Ready to go? This event is free.\r\n\r\n![stage at Atlanta Jazz Festival](https://assets3.thrillist.com/v1/image/3181942/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![stage at Atlanta Jazz Festival](https://assets3.thrillist.com/v1/image/3181942/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of Atlanta Jazz Festival### Atlanta Jazz Festival\r\n\r\n2025 dates TBDPiedmont ParkAmidst all of the innovations and creations that this country has produced, jazz music is the only true American artform\u2014and the Atlanta Jazz Festival is an event that pays homage to the genre like none other. Known as one of the largest free jazz festivals in the country, the ATL Jazz Fest is a yearly celebration of culture and art in order to expose a larger audience to the history, heritage, and variety of jazz. This event will feature artists from all over, so you will get a taste of just how eclectic this genre of music can be.Ready to go? Book here.\r\n\r\n## June\r\n\r\n### Atlanta Greek Picnic\r\n\r\n2025 dates TBDMorris Brown College\u201CIt\u2019s amazing to see fraternities and sororities from all over coming together to fellowship,\u201D says Nate Alexander, member of the Kappa Alpha Psi fraternity. Atlanta\u2019s Greek Picnic has existed as an event designed to give members of Greek lettered organizations the opportunity to fellowship and represent their sorority or fraternity. Founded in 2004, this event has attracted more than 350,000 Divine 9 members from numerous states across the country and has grown from its humble beginnings to becoming the place to reunite, socialize, network, and create memories that will last a lifetime.Ready to go? Book here.\r\n\r\n### BET Hip-Hop Awards\r\n\r\n2025 dates TBDCobb Energy Performing Arts CentreThe BET Hip Hop Awards are an annual awards show, airing on BET, showcasing hip hop performers, producers and music video directors. Taking place in Atlanta, the weekend features an array of events and activations, making the city boom with excitement. \u201CIt\u2019s a great place to network and meet people in the industry,\u201D Campbell says. \u201CPeople in high places are really accessible, too.\u201DReady to go? Book here.\r\n\r\n## July\r\n\r\n### Peachtree Road Race\r\n\r\nJuly 4, 2025Lenox SquareEvery Fourth of July weekend, the Peachtree Road Race happens at Lenox Square. \u201CThis is one of the city\u2019s oldest traditions,\u201D says Myles Braxton, an Atlanta native. \u201CIt\u2019s a good way to kick off the holiday, and it\u2019s also an exciting and communal experience.\u201D As a wellness event for all ages, the race is the perfect event for the entire family to enjoy.Ready to go? Book here.\r\n\r\n![racks of clothing at The Streetwear Market atlanta](https://assets3.thrillist.com/v1/image/3181943/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![racks of clothing at The Streetwear Market atlanta](https://assets3.thrillist.com/v1/image/3181943/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of The Streetwear Market### The Streetwear Market\r\n\r\nJuly 20\u201321, 2024Atlanta Expo CenterThe Atlanta Street Wear Market, which usually happens in April, is back for a new and exciting summer installment. This time around, the event will feature more than 200 brands, thousands of creators, and live entertainment to cap off an immersive experience for countless fashion enthusiasts throughout the city. The Summer \u201824 ASWM is taking place at the Atlanta Expo Center\u2019s 100,000-square-foot floor and will highlight a diverse range of vendors to purchase from, as well as DJ sets, free giveaways, and food selections.Ready to go? Book here.\r\n\r\n## August\r\n\r\n### Coca-Cola Summer Film Festival\r\n\r\nAugust 23\u201325, 2024The Fox TheatreEvery August, the Fox Theatre pays homage to its history as an iconic movie venue with its annual Coca Cola Summer Film Festival. This year\u2019s edition features several anniversary titles that include The Little Mermaid, Bohemian Rhapsody, Casablanca, West Side Story, and more. \u201CI look forward to the festival every year,\u201D says Mason Jones, a frequent attendee of the festival. \u201CI\u2019ve seen films like Ghostbusters and Do The Right Thing here, and the Fox never disappoints.\u201DReady to go? Book here.\r\n\r\n### Dragon Con\r\n\r\nAugust 29\u2013September 2, 2024Westin Peachtree PlazaSince 1987, Dragon Con has been a staple in Atlanta\u2019s history, now it\u2019s back for another year, again focusing on science fiction and fantasy, gaming, comics, literature, art, music, and film. Throughout the weekend, attendees can experience a series of events including burlesque shows, author signings, live performances, wrestling, and a DJ, among so many other activities. As the largest pop culture convention, it\u2019s definitely the best place if you\u2019re a fan of the genre.Ready to go? Book here.\r\n\r\n## September\r\n\r\n### Sweet Auburn Music Festival\r\n\r\nSeptember 28\u201329, 2024Old Fourth WardOne of the city\u2019s most anticipated events is back in September. The Sweet Auburn Music Fest is the perfect blend of music, food, entertainment, and culture. \u201CThe festival has some great music and performers, but it\u2019s also a celebration of the area\u2019s history,\u201D says Royale Campbell, entertainer and longtime Atlanta resident. \u201CIt\u2019s like you\u2019re getting the best of both worlds here.\u201D From the art to the live shows, there\u2019s no other music festivals in Atlanta comparable to this amazing experience.Ready to go? This event is free.\r\n\r\n### Atlanta Art Week\r\n\r\nSeptember 30\u2013October 6, 2024Founded by Art Advisor Kendra Walker in 2022 with SUTTON, Atlanta Art Week has created a dynamic environment that brings together galleries, artists, collectors, and art enthusiasts to promote cultural exchange, education, economic development, and art appreciation to the great city of ATL. \u201CAtlanta Art Week has really changed the way that the country looks at this city\u2019s art scene,\u201D says Jaleesa Taylor, a local event curator. \u201CIt\u2019s really important that this continues to grow.\u201DReady to go? Book here.\r\n\r\n## October\r\n\r\n![parade for Atlanta Pride](https://assets3.thrillist.com/v1/image/3181947/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![parade for Atlanta Pride](https://assets3.thrillist.com/v1/image/3181947/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of WABE and Atlanta Pride### Atlanta Pride\r\n\r\nOctober 11\u201313, 2024Various locationsAtlanta Pride is back again for its annual celebration of the LGBTQ\u002B community. This year, attendees will be able to experience an array of events, such as the Kickoff Party at the Georgia Aquarium, the United with Pride Music Festival at Piedmont Park, dozens of marches, and more. The weekend culminates on Sunday afternoon with the Atlanta Pride Parade, which will begin at the Civic Center MARTA Station. As always, the festival and parade are free of charge, so come out and have a great time for an even better cause.Ready to go? This event is free.\r\n\r\n### Spelhouse Homecoming \u0026amp; Tailgate\r\n\r\nOctober 26, 2024Morehouse CollegeThere are many things to look forward to during the fall season in Atlanta, and the Spelhouse tailgate is one of those things that sits at the top of the list. Morehouse College\u2019s campus will be filled with thousands of people looking to eat, drink, and kick back with people from across the country. As well as being a time for students and alums to gather, homecoming is also a celebration of HBCU culture and shared experiences, so it\u2019s something that everyone should attend at least once in life.Ready to go? This event is free.\r\n\r\n### ONE Musicfest\r\n\r\nOctober 26\u201327, 2024Piedmont ParkEvery time I\u2019ve gone to ONE Musicfest, it has reminded me of a big family reunion. I tend to bump into people I haven\u2019t seen in a long time, and the atmosphere is always filled with love. On top of that, the performers are second-to-none. Artists such as Janet Jackson, Kendrick Lamer, Pharrell, Outkast, Usher, Lauryn Hill, SZA, Nas, Erykah Badu, and so many others have graced the OMF stage, making it one of the most diverse festivals in the country.Ready to go? Book here.\r\n\r\n![ferris wheel at night at The Atlanta Fair](https://assets3.thrillist.com/v1/image/3181948/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\n![ferris wheel at night at The Atlanta Fair](https://assets3.thrillist.com/v1/image/3181948/381x254/crop;webp=auto;jpeg_quality=60.jpg)\r\nPhoto courtesy of The Atlanta Fair### The Atlanta Fair\r\n\r\nOctober 4\u2013November 3, 2024Old Turner FieldThe Atlanta Fair returns to Center Parc Stadium\u2014formerly Turner Field\u2014on October 4. Standing as Georgia\u2019s only urban fair, this event will include all your fairground favorites, including deep-fried Oreos and Snickers, funnel cakes and more. The Atlanta Fair also features some exciting rides, and yields an amazing view of Downtown if you decide to hop on the heralded Ferris Wheel. For more than 50 years the fair has dazzled attendees from near and far, so you can bet your bottom dollar that this year will be just as satisfying as the last.Ready to go? Book here.\r\n\r\n## December\r\n\r\n### SEC Championship\r\n\r\nDecember 7, 2024Mercedes-Benz StadiumThe SEC Championship Game has been held in the ATL since 1994, and it features the top two teams in the NCAA\u2019s Southeastern Conference. Known as the toughest conference in all of college football, squads such as Georgia, LSU, Alabama and Florida have all grabbed the trophy in years past. \u201CSo many times the winner of this game goes on to snag the National Championship,\u201D Atlanta native Everett Jolly says. \u201CSo it\u2019s must-see TV from start to finish.\u201DReady to go? Book here.\r\n\r\nWant more Thrillist? Follow us on Instagram, TikTok, Twitter, Facebook, Pinterest, and YouTube.\r\n\r\n[Okla Jones](https://www.thrillist.com/authors/okla-jones)*is an Atlanta-based journalist who writes about food, fine arts, and entertainment. His work also appears in*ESSENCE*,*Creative Loafing Atlanta*, and*Consequence of Sound*. Follow him on Instagram at*[@coolhandoak](http://www.instagram.com/coolhandoak)*.*By signing up, I agree to the[Terms](https://www.voxmedia.com/legal/terms-of-use)and[Privacy Policy.](https://www.voxmedia.com/legal/privacy-notice)* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n\r\n* Cookie Settings\r\n* Ethics Statement\r\n* Community Guidelines\r\n* How to Pitch\r\n* \r\n* About Us\r\n* Newsletter\r\n* Careers\r\n* Advertise With Us\r\n* Press\r\n* Accessibility\r\n* Privacy\r\n* Cookie Policy\r\n* Terms \u002B Conditions\r\n\r\n[](https://voxmedia.com)This publication is a Vox Media network.\u00A92024[Vox Media](https://voxmedia.com), LLC. All Rights Reserved.[](/)[Subscribe](/newsletters)* CitiesAtlantaAustinBostonChicagoDallasDenverHoustonLas VegasLos AngelesMiamiNashvilleNew OrleansNew YorkPhiladelphiaPhoenixSan DiegoSan FranciscoWashington DC\r\n* Plan Your TripTips \u0026amp; HacksWhat to PackLodgingFlightsTrainsCruisesRoad TripsBudgetSplurge\r\n* Guides \u0026amp; DestinationsNeighborhood GuidesMapsTravel ItinerariesTravel InspirationWeekend GetawaysBig CitiesSmall TownsHidden GemsInternational TripsMountainsDesertsForestsBeaches\r\n* ExperiencesArts \u0026amp; CultureCannabisFamilyFood \u0026amp; DrinkHikingLGBTQ\u002BMusicNightlifeOutdoors \u0026amp; NatureShopping \u0026amp; StyleSportsStargazingWellness\r\n* News \u0026amp; TrendsNewsTrends\r\n* Video\r\n\r\nSubscribe to Thrillist DailyAdventure is always around the corner. And now it\u2019s also in your inbox!SubscribeBy signing up, I agree to the[Terms](https://www.voxmedia.com/legal/terms-of-use)and[Privacy Policy.](https://www.voxmedia.com/legal/privacy-notice)* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n\r\n* Cookie Settings\r\n* Ethics Statement\r\n* Community Guidelines\r\n* How to Pitch\r\n* \r\n* About Us\r\n* Newsletter\r\n* Careers\r\n* Advertise With Us\r\n* Press\r\n* Accessibility\r\n* Privacy\r\n* Cookie Policy\r\n* Terms \u002B Conditions\r\n\r\n[](https://voxmedia.com)\u00A92024[Vox Media](https://voxmedia.com), LLC.All Rights Reserved."
```

Here are some exciting things to do in Atlanta today, October 3, 2024:

1. **Charli XCX & Troye Sivan Concert**
   - **Time:** 7:30 PM
   - **Location:** State Farm Arena, One Philips Drive, Atlanta, GA
   - **Details:** Enjoy a night with pop sensations Charli XCX and Troye Sivan.
   - ![Charli XCX & Troye Sivan](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)
   - [More Info & Tickets](https://atlanta-ga.events/tickets/charli-xcx/)

2. **The Lion King Musical**
   - **Time:** 7:30 PM
   - **Location:** Fabulous Fox Theatre, 660 Peachtree St NE, Atlanta, GA
   - **Details:** Experience the beloved Disney musical live on stage.
   - ![The Lion King](https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1)
   - [More Info & Tickets](https://atlanta-ga.events/tickets/the-lion-king/)

3. **Atlanta Falcons vs. Tampa Bay Buccaneers**
   - **Time:** 8:15 PM
   - **Location:** Mercedes-Benz Stadium, Martin Luther King Jr Dr SW & Northside Dr NW, Atlanta, GA
   - **Details:** Watch the thrilling NFL game between the Falcons and the Buccaneers.
   - [More Info & Tickets](https://atlanta-ga.events/tickets/atlanta-falcons/)

4. **Ziggy Marley and Marley Brothers Concert**
   - **Time:** 7:30 PM
   - **Location:** Cellairis Amphitheatre at Lakewood, 2002 Lakewood Avenue Southeast, Atlanta, GA
   - **Details:** Enjoy a reggae night with Ziggy Marley and his brothers.
   - [More Info & Tickets](https://atlanta-ga.events/tickets/ziggy-marley/)

5. **Inaugural Atlanta Art Fair**
   - **Date:** October 3-6, 2024
   - **Location:** Pullman Yards, five miles east of downtown Atlanta
   - **Details:** Explore the inaugural Atlanta Art Fair, coinciding with Atlanta's third annual Art Week.
   - ![Atlanta Art Fair](https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6&w=80&h=80&c=1&pid=5.1)
   - [More Info](https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/)

These events provide a variety of entertainment options for you to enjoy in Atlanta today. For more details and tickets, please refer to the provided links.

![Atlanta](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

*Sources:*
- [Atlanta Events October 2024](https://atlanta-ga.events/october/)
- [Go South Atlanta Events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
- [Songkick Atlanta Events](https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024)
- [Forbes Atlanta Art Fair](https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/)
- [Thrillist Atlanta Events](https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year)


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting things to do in Atlanta today, October 3, 2024:

1. **Charli XCX & Troye Sivan Concert**
   - **Time:** 7:30 PM
   - **Location:** State Farm Arena, One Philips Drive, Atlanta, GA
   - **Details:** Enjoy a night with pop sensations Charli XCX and Troye Sivan.
   - ![Charli XCX & Troye Sivan](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)
   - [More Info & Tickets](https://atlanta-ga.events/tickets/charli-xcx/)

2. **The Lion King Musical**
   - **Time:** 7:30 PM
   - **Location:** Fabulous Fox Theatre, 660 Peachtree St NE, Atlanta, GA
   - **Details:** Experience the beloved Disney musical live on stage.
   - ![The Lion King](https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1)
   - [More Info & Tickets](https://atlanta-ga.events/tickets/the-lion-king/)

3. **Atlanta Falcons vs. Tampa Bay Buccaneers**
   - **Time:** 8:15 PM
   - **Location:** Mercedes-Benz Stadium, Martin Luther King Jr Dr SW & Northside Dr NW, Atlanta, GA
   - **Details:** Watch the thrilling NFL game between the Falcons and the Buccaneers.
   - [More Info & Tickets](https://atlanta-ga.events/tickets/atlanta-falcons/)

4. **Ziggy Marley and Marley Brothers Concert**
   - **Time:** 7:30 PM
   - **Location:** Cellairis Amphitheatre at Lakewood, 2002 Lakewood Avenue Southeast, Atlanta, GA
   - **Details:** Enjoy a reggae night with Ziggy Marley and his brothers.
   - [More Info & Tickets](https://atlanta-ga.events/tickets/ziggy-marley/)

5. **Inaugural Atlanta Art Fair**
   - **Date:** October 3-6, 2024
   - **Location:** Pullman Yards, five miles east of downtown Atlanta
   - **Details:** Explore the inaugural Atlanta Art Fair, coinciding with Atlanta's third annual Art Week.
   - ![Atlanta Art Fair](https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6&w=80&h=80&c=1&pid=5.1)
   - [More Info](https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/)

These events provide a variety of entertainment options for you to enjoy in Atlanta today. For more details and tickets, please refer to the provided links.

![Atlanta](https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1)

*Sources:*
- [Atlanta Events October 2024](https://atlanta-ga.events/october/)
- [Go South Atlanta Events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
- [Songkick Atlanta Events](https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024)
- [Forbes Atlanta Art Fair](https://www.forbes.com/sites/chaddscott/2024/10/01/inaugural-atlanta-art-fair-taking-place-october-3-6-at-pullman-yards/)
- [Thrillist Atlanta Events](https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year)

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Calculate the dates for the next four weekends
weekend_dates = [(today + timedelta(days=(5 - today.weekday() + i * 7))) for i in range(4)]

# Output the dates for the next four weekends
weekend_dates
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on October 5, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta on October 5, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4907344191957257\u0026mkt=en-US\u0026setlang=en-US\u0026w=gk2PwJ4MGAK3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"Things to do this weekend in metro Atlanta | Oct. 5-7, 2024 - Yahoo","url":"https://www.yahoo.com/lifestyle/things-weekend-metro-atlanta-oct-202707494.html","displayUrl":"https://www.yahoo.com/lifestyle/things-weekend-metro-atlanta-oct-202707494.html","snippet":"When: 10 a.m. to 5 p.m. Oct. 5 and 11 a.m. to 5 p.m. Oct. 6 Where: Olmstead Linear Park, Atlanta What: Arts and crafts festival with over 125 displays, food, and a children\u0026#39;s area. Admission: Free ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=2314347778720\u0026mkt=en-US\u0026setlang=en-US\u0026w=0Bc1_sKUVRUQP4fhy1T4L9cQiKL3UoSF"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4714289709671445\u0026mkt=en-US\u0026setlang=en-US\u0026w=uP6_4hpqL9Zfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 5 and 6, 2024 Candler Park Atlanta FREE. This popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more. ... October 12 and 13, 2024 Piedmont Park. An annual October event! Atlanta Pride features a festival and parade in Midtown ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=5012794232883316\u0026mkt=en-US\u0026setlang=en-US\u0026w=IFPPoHWFm5znjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Las nubes, D\u00E1vila 666, and Ladrones (US) The Star Community Bar, Atlanta, GA, US. Thursday 03 October 2024. Charli XCX and Troye Sivan. State Farm Arena, Atlanta, GA, US. Ziggy Marley and Marley Brothers Stephen Marley, Julian Marley, Ky-mani Marley, and Damian \u0026#39;Jr Gong\u0026#39; Marley. Cellairis Amphitheatre at Lakewood, Atlanta, GA, US.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4726551834592006\u0026mkt=en-US\u0026setlang=en-US\u0026w=77Xe8SkyBFllYyuTqLMEVsVddRXByGfo"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4520195834871245\u0026mkt=en-US\u0026setlang=en-US\u0026w=sqN1N4yyzZUHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids","url":"https://mommypoppins.com/atlanta-kids/weekend-events/best-things-to-do-in-atlanta-in-october-halloween-weekend","displayUrl":"https://mommypoppins.com/atlanta-kids/weekend-events/best-things-to-do-in-atlanta-in...","snippet":"Kick off October events in Atlanta with fried food, amazing rides, ... Atlanta - FREE! October 5 \u0026amp; 6, 2024 . Families check out 125 displays of arts and crafts (think folk and outsider style) at historic Olmsted Linear Park. Besides the main attraction, visitors can explore the children\u0026#39;s area, try local food and beverages, and listen to ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=1833269792522\u0026mkt=en-US\u0026setlang=en-US\u0026w=2WgIM99orhoqHTTsGTDReINgTpcgCSGt"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Search the Atlanta events calendar below or explore our annual festivals and seasonal events taking place throughout the year. ... October 10/5/24. Rejuvenate Your Body, Reclaim Your Mind 10/5/24. Rise and Dine with the Animals: Gorillas ... Venture Atlanta 2024 10/8/24 \u2013 10/9/24. Atlanta miniFEST 10/9/24 \u2013 10/10/24. Geology Walk and Talk ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4819881477614541\u0026mkt=en-US\u0026setlang=en-US\u0026w=cQIKbv56z1uP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don\u0026#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=5050860520874015\u0026mkt=en-US\u0026setlang=en-US\u0026w=fS7GwpbIIKf43ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on October 12, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta on October 12, 2024"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 12:00 PM Fri. Days of the Dead - Friday The Courtland Grand Hotel. 30303, 165 Courtland Street NE, Atlanta, GA, US Prices from $69 Avg. price ~ $79. 2 tickets remaining! ..."},{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"\u2013 Joyce Meyer Live, October 11-12 2024 Faith event at the Gas South Arena. More details \u2013 Take Your Royal Position Conference, October 11-12 2024 ... \u2013 Atlanta Pride Festival, October 12-13 2024 One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more. ..."},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids - MSN","url":"https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","displayUrl":"https://www.msn.com/.../33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","snippet":"October 12 \u0026amp; 13; 19 \u0026amp; 20, 2024 Check out the crispiest, juiciest apples from Georgia\u0026#39;s apple capital--Ellijay. For two weekends, Ellijay hosts the Georgia Apple Festival complete with arts and ..."},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Jeff Lynne\u0026#39;s ELO - State Farm Arena","url":"https://www.statefarmarena.com/events/detail/jeff-lynnes-electric-light-orchestra-elo-the-over-and-out-tour","displayUrl":"https://www.statefarmarena.com/events/detail/jeff-lynnes-electric-light-orchestra-elo...","snippet":"JEFF LYNNE\u2019S ELO ANNOUNCES 2024 NORTH AMERICAN FALLTOUR, FEATURING THEIR FINAL STOP IN ATLANTA AT STATE FARM ARENA ON SATURDAY, OCT. 12. \u201CTHE OVER AND OUT TOUR\u201D WILL BE THE BAND\u2019S FINAL TOUR. GENERAL ONSALE BEGINS MARCH 22 @ 10 AM LOCAL TIME HERE AND TICKETMASTER.COM. (March 18, 2024) \u2013 Jeff Lynne\u2019s ELO is set to embark on the band ...","thumbnailUrl":"https://www.bing.com/th?id=OIP._u1L0RmxvdGbXBflp5bo1AHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Multiple dates until 10/12/2024. Roberta Lea Live 10/5/24. Strand Ole Opry: Fa-BOO-lous Halloween Hits 10/5/24. The Modern x Atlas ... October 12. Saturday. Sangria n\u2019 Skincare DIY Facial Serum Workshop ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Gucci Mane: The Road to 1017 - Atlanta Symphony Orchestra","url":"https://www.aso.org/events/detail/gucci-mane","displayUrl":"https://www.aso.org/events/detail/gucci-mane","snippet":"Mark your calendars for October 12, 2024, and join us at the Atlanta Symphony Hall for a night that promises to be a legendary moment in music. ... Chorus member Tierney Breedlove will be dropping by on Instagram tonight to share some stories from the event--check in later to see what\u0026#39;s happening. If you\u0026#39;d like to share your own story, images ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.AMflb5sDJwvsNctioqnIsQHaE3\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 Upcoming Festivals in Atlanta 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","snippet":"November 2024. \u2013 Native American Festival and Pow Wow, October 31 - November 3 2024. Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details.","thumbnailUrl":"https://www.bing.com/th?id=OIP.1kWLxDO_JldjYbZyTlfSrQAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on October 19, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta on October 19, 2024"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. ... 19 tickets remaining! Tickets; Oct. 02. 2024. 7:00 PM Wed. Dayseeker The Eastern - GA. 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $47 Avg. price ~ $80 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"\u2013 Politics War Room with James Carville \u0026amp; Al Hunt, October 12 2024 Talk event at Variety Playhouse. More details \u2013 Atlanta Pride Festival, October 12-13 2024 ... \u2013 PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4907344191957257\u0026mkt=en-US\u0026setlang=en-US\u0026w=gk2PwJ4MGAK3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Concerts in Atlanta in October. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 581 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Sat 19 Oct 2024 Boot Barn Hall at Bourbon Brothers Gainesville, GA, US. Lauren Sanderson ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4726551834592006\u0026mkt=en-US\u0026setlang=en-US\u0026w=77Xe8SkyBFllYyuTqLMEVsVddRXByGfo"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=5012794232883316\u0026mkt=en-US\u0026setlang=en-US\u0026w=IFPPoHWFm5znjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4520195834871245\u0026mkt=en-US\u0026setlang=en-US\u0026w=sqN1N4yyzZUHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Trivia Night at Wild Leap Atlanta Every Tuesday until 11/19/2024. Elements- Surya Ensemble 10/2/24 \u2013 10/10/24 ... October 19. Saturday. Little 5 Points Halloween Festival \u0026amp; Parade 10/19/24 \u2013 10/20/24 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4819881477614541\u0026mkt=en-US\u0026setlang=en-US\u0026w=cQIKbv56z1uP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in 2024 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2024","snippet":"Concerts in Atlanta in 2024. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 1216 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Sat 19 Oct 2024 Boot Barn Hall at Bourbon Brothers Gainesville, GA, US. Lauren Sanderson ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4943520702678011\u0026mkt=en-US\u0026setlang=en-US\u0026w=qk9MCkyzQ_7IDd8Wa8glqjNaLOrIMgoZ"},{"name":"Georgia Tech vs. Notre Dame, October 19, 2024 | Event at Mercedes-Benz ...","url":"https://www.mercedesbenzstadium.com/events/georgia-tech-vs-notre-dame","displayUrl":"https://www.mercedesbenzstadium.com/events/georgia-tech-vs-notre-dame","snippet":"Specialty and premium food offerings, custom developed culinary concepts, and unique food and beverage destinations are available on all levels of the stadium. Atlanta, GA | Experience the thrill of college football as Georgia Tech hosts Notre Dame at Mercedes-Benz Stadium on October 19th. Feel the excitement and school spirit in this epic ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4730662125454896\u0026mkt=en-US\u0026setlang=en-US\u0026w=UeLdSV5RI4tnQcVeEQPoW-0EdnNjCR7N"},{"name":"Concerts In Atlanta, October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","snippet":"Concerts On Friday October 11 2024. \u2013 Lake Street Dive at The Eastern. More details. \u2013 Aerosmith: Peace Out The Farewell Tour at State Farm Arena. More details. \u2013 The Black Jacket Symphony: Performing Led Zeppelin\u2019s \u2018IV\u2019. Tribute concert at Atlanta Symphony Hall. More details. \u2013 Loveless at The Masquerade.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4991061696394872\u0026mkt=en-US\u0026setlang=en-US\u0026w=dZPhq6ZRLkrdrEf4HxlkWHNc_Lnct5jK"},{"name":"Upcoming Events - Discover Atlanta","url":"https://discoveratlanta.com/events/all/","displayUrl":"https://discoveratlanta.com/events/all","snippet":"Bounce through the incredible events happening around Atlanta today, tomorrow, this week or this month. Plan your fun and be in the know on everything from family-favorites, to can\u2019t-miss hot spots, concerts and foodie festivals. Showing 1 \u2013 15 of 484. Editors\u2019 Pick.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4829094188486860\u0026mkt=en-US\u0026setlang=en-US\u0026w=muOUpmxa0vCUBKZlUeD3eG4MmVcBwK29"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on October 26, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta on October 26, 2024"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"\u2013 A Conversation with Larry David, October 24 2024 Talk event at the Fox Theatre. More details \u2013 UpTempo \u2013 A Composer\u2019s Voice: From the Serene to the Terrifying, October 24 2024 ... October 26-27 2024 Air show at Atlanta Regional Airport, Peachtree City. More details \u2013 ONE Musicfest, October 26-27 2024 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4907344191957257\u0026mkt=en-US\u0026setlang=en-US\u0026w=gk2PwJ4MGAK3mkDLUvcm_m1afK8iBTpt"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4714289709671445\u0026mkt=en-US\u0026setlang=en-US\u0026w=uP6_4hpqL9Zfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Sneaker Con Atlanta Event - October 26, 2024","url":"https://sneakercon.com/event/sneaker-con-atlanta-october-26-2024/","displayUrl":"https://sneakercon.com/event/sneaker-con-atlanta-october-26-2024","snippet":"October 26, 2024. Cobb Galleria Centre. 2 Galleria Pkwy SE. Atlanta, GA 30339. Become a vendor. Book Atlanta Hotel. Celebrate Sneaker Con\u2019s 15th Anniversary in Atlanta on October 26, 2024. We\u2019re back with fresh new artwork, a bigger and better show, and more sneakers than you can dream of\u2014all under one roof at the Cobb Galleria Centre.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4674668634919930\u0026mkt=en-US\u0026setlang=en-US\u0026w=sPDB0xT7df1NyxSkpqSAB4bae5GbQT1Y"},{"name":"The 15th Annual ONE Musicfest in Atlanta, GA | Oct. 26-27, 2024","url":"https://concerts.theurbanmusicscene.com/2024/08/the-15th-annual-one-musicfest/","displayUrl":"https://concerts.theurbanmusicscene.com/2024/08/the-15th-annual-one-musicfest","snippet":"Festival Dates are October 26th \u0026amp; 27th at Piedmont Park in Atlanta, GA. ATLANTA, Aug. 9, 2024 /PRNewswire/ \u2014 ONE Musicfest (OMF), presented by Procter and Gamble (P\u0026amp;G), in collaboration with Live Nation Urban (LNU), is excited to announce its eagerly awaited lineup for its 15th anniversary of music, culture, and celebration.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4944985283231748\u0026mkt=en-US\u0026setlang=en-US\u0026w=6LzJ_FlpO0jIuAAWCGuuzkLnTkhV7pbQ"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Las nubes, D\u00E1vila 666, and Ladrones (US) The Star Community Bar, Atlanta, GA, US. Thursday 03 October 2024. Charli XCX and Troye Sivan. State Farm Arena, Atlanta, GA, US. Ziggy Marley and Marley Brothers Stephen Marley, Julian Marley, Ky-mani Marley, and Damian \u0026#39;Jr Gong\u0026#39; Marley. Cellairis Amphitheatre at Lakewood, Atlanta, GA, US.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4726551834592006\u0026mkt=en-US\u0026setlang=en-US\u0026w=77Xe8SkyBFllYyuTqLMEVsVddRXByGfo"},{"name":"ONE MusicFest Festival Guide - Discover Atlanta","url":"https://discoveratlanta.com/events/one-musicfest/","displayUrl":"https://discoveratlanta.com/events/one-musicfest","snippet":"Non-stop music and festival fun all weekend long. When: October 26, 2024 \u2013 October 27, 2024. Where: Piedmont Park. Celebrating another year in Atlanta, One MusicFest stays true to its name by bringing together a diverse range of artists hailing from music genres such as old school to today\u2019s hip-hop, R\u0026amp;B, house, trap beats, soul, and more.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4591204547254456\u0026mkt=en-US\u0026setlang=en-US\u0026w=7EVDeaK11JIn1uRwW6F9IzpgYURs9noI","thumbnailUrl":"https://www.bing.com/th?id=OIP.lPUU4EMfH7nb4_X6e4I4UwHaFS\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"September 27th to October 6, 2024 Atlanta Motor Speedway ... October 12 and 13, 2024 Piedmont Park. An annual October event! Atlanta Pride features a festival and parade in Midtown, plus a wide variety of associated events around town. ... October 26, 2024 Oakhurst Village FREE.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=5012794232883316\u0026mkt=en-US\u0026setlang=en-US\u0026w=IFPPoHWFm5znjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta October 2024 Events Atlanta October 2024 Events Destinations Atlanta National events. Search events nationwide. Select Cities. ... Oct 26-27, 2024 Piedmont Park View on Map. Hold onto your headphones, folks. One Musicfest, the big daddy of all 2-day urban music bashes, is back with a star-studded lineup. Cardi B and Gunna will be taking ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=5050860520874015\u0026mkt=en-US\u0026setlang=en-US\u0026w=fS7GwpbIIKf43ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Thursday until 12/26/2024. Sunset Sessions Presents DJ Mike Webster \u0026amp; DJ Xzac 10/2/24 \u2013 10/13/24. ... October 26. Saturday. 18th PF HALLOWEEN PARTY \u0026amp; COSTUME CONTEST 10/26/24 \u2013 10/27/24. October 26.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B26%2c\u002B2024\u0026d=4819881477614541\u0026mkt=en-US\u0026setlang=en-US\u0026w=cQIKbv56z1uP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html"}
```
and got this result:
```
"Atlanta Events - October 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; October Events\r\n\r\n# Atlanta Events - October 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Concerts in Atlanta in October \u0026ndash; Atlanta\u0026rsquo;s biggest annual festivals \u0026ndash; Atlanta weather in October \u0026ndash; Things to do in Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From October 1 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; See September events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center with tournament prizes across 20 gaming contests. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; OYE Fest, October 6 2024Annual festival in Historic Fourth Ward Park celebrating Latinx art and music. More details\r\n\r\n\u0026ndash; La Cotorrisa Tour, October 6 2024Spanish-language comedy podcast show at the Buckhead Theatre. More details\r\n\r\n## Events From October 7 To Columbus Day Weekend, October 12-14 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Atlanta miniFEST, October 9 2024Community music and arts festival at Believe Music Hall. More details\r\n\r\n\u0026ndash; Taste Brookhaven, October 10 2024Food-tasting event in the gardens of the Hyatt Regency Atlanta Perimeter, featuring Brookhaven chefs and restaurants plus drinks and live entertainment, to benefit Children\u0026rsquo;s Healthcare of Atlanta. More details\r\n\r\n\u0026ndash; Jake Shane, October 10 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Scott Antique Market, October 10-13 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; Atsuko Okatsuka, October 11 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Pride Kickoff, October 11 2024After-hours LGBTQ\u002B party at the Georgia Aquarium. More details\r\n\r\n\u0026ndash; Wire \u0026amp; Wood Alpharetta, October 11-12 2024Songwriter music festival with performances across downtown Alpharetta. More details\r\n\r\n\u0026ndash; Joyce Meyer Live, October 11-12 2024Faith event at the Gas South Arena. More details\r\n\r\n\u0026ndash; Take Your Royal Position Conference, October 11-12 2024Faith and empowerment event in downtown Atlanta. More details\r\n\r\n\u0026ndash; Duluth Fall Home Expo, October 11-13 2024Home renovations trade show at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; Oakhurst Porchfest, October 12 2024Annual community music festival in Decatur over 100 live performances on Oakhurst neighborhood porches. More details\r\n\r\n\u0026ndash; Madison Avenue Derby, October 12 2024Annual races in Decatur in homemade, human-powered soapbox cars, to benefit local children\u0026rsquo;s charities. More details\r\n\r\n\u0026ndash; Politics War Room with James Carville \u0026amp; Al Hunt, October 12 2024Talk event at Variety Playhouse. More details\r\n\r\n\u0026ndash; Atlanta Pride Festival, October 12-13 2024One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more. More details\r\n\r\n\u0026ndash; Taste \u0026amp; Brews Fall Festival, October 12-13 2024Annual food and beer festival at Etowah River Park, Canton, featuring southern food and beers, musicians from across Georgia, arts and crafts, farmers market, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Old Fourth Ward Fall Arts Festival, October 12-13 2024Twice-annual arts festival in Historic Fourth Ward Park featuring local and Georgian artists, demonstrations, live entertainment, art activities, food and more. More details\r\n\r\n\u0026ndash; Chalktoberfest, October 12-13 2024Popular annual art and beer festival in Marietta Square, featuring professional chalk artists, 120\u002B craft beers, live music, arts and crafts, food trucks, amateur chalk art competition, and more. More details\r\n\r\n\u0026ndash; FashionFest, October 12-13 2024Festival of urban-popular fashion and streetwear at Ambient Plus Studio, with fashion and footwear vendors, food, music and more. More details\r\n\r\n\u0026ndash; Gold Over America Tour, October 13 2024Gymnastics performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Roswell Town Square Dinner Series, October 13 2024Outdoor community dining event at Roswell Town Square with food from The Mill Kitchen \u0026amp; Bar. More details\r\n\r\n\u0026ndash; Pride Parade Viewing Party, October 13 2024Rooftop LGBTQ\u002B party at the Fox Theatre, to coincide with the annual Pride parade. More details\r\n\r\n\u0026ndash; Joe Wong, October 14 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n## Events From October 15 To The Weekend Of October 19-20 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; David Cross \u0026ndash; End of the Beginning of the End, October 16 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jessica Kirson, October 18 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Sharon McMahon, October 18 2024Author talk at Center Stage. More details\r\n\r\n\u0026ndash; Stone Mountain Highland Games and Scottish Festival, October 18-20 2024Annual festival of Scottish heritage at Stone Mountain Park, with traditional Scottish athletics, music, dancing, food and drink, and vendors. All are welcome. More details\r\n\r\n\u0026ndash; Dracula: Presented by Gwinnett Ballet Theatre, October 18-20 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Georgia Metropolitan Dance Theatre presents Dracula, October 18-20 2024Ballet performance at the KSU Dance Theatre, Marietta Campus. More details\r\n\r\n\u0026ndash; A Taste of Chamblee, October 19 2024Annual food festival in downtown Chamblee, with food from neighborhood restaurants, craft vendors, children\u0026rsquo;s activities, football screenings, live performances, wine tent, beer garden and more. More details\r\n\r\n\u0026ndash; HarvestFest and Scarecrows In The Square, October 19 2024Annual fall festival in Marietta Square, with arts and crafts show, touch-a-truck, scarecrow displays, costume and pie eating contests, and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Boo! Halloween, October 19 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; Cherokee Heights Arts Festival, October 19 2024Free annual arts festival in the Cherokee Heights neighborhood, with local artists and makers, local musicians, art activities, food, beer and more. More details\r\n\r\n\u0026ndash; NBAF Celebration Under The Stars, October 19 2024Fundraising event for the National Black Arts Festival at Wolf Creek Amphitheater, with live performances and food and drink. More details\r\n\r\n\u0026ndash; Rickey Smiley, October 19 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Carlos Ballarta: Tlatoani, October 19 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Witch Perfect, October 19 2024Drag performance at Center Stage. More details\r\n\r\n\u0026ndash; L5P Halloween Festival and Parade, October 19-20 2024Popular annual Halloween celebration in Little Five Points, with live music, parade, haunted house, ghost tours and more. More details\r\n\r\n\u0026ndash; PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. More details\r\n\r\n\u0026ndash; Johns Creek Arts Festival, October 19-20 2024Annual arts and crafts festival in Johns Creek featuring 140 artisans from across the country. More details\r\n\r\n\u0026ndash; Abnormal Market and Uncommon Bazaar, October 19-20 2024Art market in association with the Old Fourth Ward Park Arts Festival. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Landlocked 8, October 20 2024Oyster and BBQ festival at Legacy Park, Decatur. More details\r\n\r\n\u0026ndash; MrBallen: The Strange, Dark, \u0026amp; Mysterious Live Tour, October 20 2024Podcast live show at the Coca-Cola Roxy. More details\r\n\r\n\u0026ndash; National Ballet of Ukraine, October 20 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Shakespeare Out of a Hat: Macbeth, October 20 2024Part-improvised play at the Shakespeare Tavern Playhouse. More details\r\n\r\n## Events From October 21 To The Weekend Of October 26-27 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Malcolm Gladwell in conversation with Virginia Prescott \u0026ndash; Revenge of the Tipping Point, October 22 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Jim Henson\u0026rsquo;s Labyrinth: In Concert, October 23 2024Film screening with live musical accompaniment at The Eastern. More details\r\n\r\n\u0026ndash; A Conversation with Larry David, October 24 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; UpTempo \u0026ndash; A Composer\u0026rsquo;s Voice: From the Serene to the Terrifying, October 24 2024Pre-concert event at Woodruff Arts Center aimed at teens and their families. More details\r\n\r\n\u0026ndash; Whitney Cummings: Big Baby, October 25 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Midtown Halloween Block Party, October 25 2024Halloween-themed block party and costume contest at Politan Row at Colony Square. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part I, October 25 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Benteen Park Fall Fest, October 26 2024Annual fall festival in Benteen Park, with live music, artist market and food vendors. More details\r\n\r\n\u0026ndash; Yappy Hour, October 26 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; Jerry Seinfeld, October 26 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Felipe Esparza, October 26 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Halloween in the Highland, October 26 2024Halloween-themed block party and costume contest in the Virginia-Highland neighborhood. More details\r\n\r\n\u0026ndash; Buckhead Creepy Crawl, October 26 2024Halloween-themed bar crawl and costume contest in the Buckhead entertainment district. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Air Dot Show Tour, October 26-27 2024Air show at Atlanta Regional Airport, Peachtree City. More details\r\n\r\n\u0026ndash; ONE Musicfest, October 26-27 2024Big annual festival of urban progressive music, in Piedmont Park. More details\r\n\r\n\u0026ndash; Menopause The Musical 2: Cruising Through \u0026lsquo;The Change\u0026rsquo;, October 26-27 2024Musical at the Gas South Theater. More details\r\n\r\n\u0026ndash; CompassionCon, October 27 2024Annual community wellness festival in Legacy Park, Decatur, with art, music, wellness activities, health services, food trucks and more. More details\r\n\r\n\u0026ndash; Hillary Clinton Live, October 27 2024Author talk at the Fox Theatre. More details\r\n\r\n## Events From October 28 To THe Weekend Of November 2-3 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Historical Marker Dedication: Booker T Washington High School, October 29 2024Historical marker dedication by the Georgia Historical Society at Atlanta\u0026rsquo;s first public secondary school for Black students, at Booker T Washington High School, 45 Whitehouse Drive SW. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part II, October 31 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Native American Festival and Pow Wow, October 31 - November 3 2024Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details\r\n\r\n\u0026ndash; Two Hot Takes Podcast: No Takes Left Behind, November 1 2024Podcast live show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Black Girls Rock!, November 1 2024Annual awards ceremony at the Fox Theatre celebrating the accomplishments of Black women. More details\r\n\r\n\u0026ndash; Ghostbusters in Concert, November 1-2 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Party With Impact \u0026ndash; Positive Impact Health Centers, November 1-2 2024Comedy show at the Gas South Theater to benefit LGBTQ\u002B healthcare services in Atlanta. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, November 1-3 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Rescue Dog Games: Howl-o-Ween Festival, November 2 2024Annual pet-friendly festival in Piedmont Park, with dog activities, dog costume parade and contest, pet-centric vendors, live music, food and beer, adoptable pets, and more. More details\r\n\r\n\u0026ndash; Chomp and Stomp, November 2 2024Annual neighborhood festival in Cabbagetown, with chili, beer, bluegrass music and more, to benefit the Friends of Cabbagetown Conservancy. More details\r\n\r\n\u0026ndash; D\u0026iacute;a de los Muertos Festival, November 2 2024Free family festival at Woodruff Arts Center, with live music, mariachi band, vendors, food trucks, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Suwanee Wine Fest, November 2 2024Annual festival in Suwanee Town Center Park, with wine samples from across the world, Georgia wines, live music, arts and crafts vendors, food and more. More details\r\n\r\n\u0026ndash; \u0026ldquo;Honoring Our Past Through Stories\u0026rdquo; Historic Landscape Preservation Grant Fund Benefit, November 2 2024Outdoor event at the Marietta Educational Garden Center to benefit the Garden Club of Georgia\u0026rsquo;s Historic Landscape Preservation Grant, with speakers, plein air painters, exhibits, music and more. More details\r\n\r\n\u0026ndash; Chastain Park Arts Festival, November 2-3 2024Popular twice-annual arts festival at Chastain Park, with 175\u002B arts and crafts vendors, food trucks, local musicians, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; The Elegant Elf Marketplace, November 2-3 2024Annual holiday marketplace with 80\u002B vendors at the Sandy Springs Performing Arts \u0026amp; Conference Center, to benefit The Sandy Springs Society. More details\r\n\r\n\u0026ndash; Jeff Arcuri: The Full Beans Fall Tour, November 3 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Disney Jr Live On Tour: Let\u0026rsquo;s Play, November 3 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026rarr; See November events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/october/"}
```
and got this result:
```
"Atlanta Events October 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* October\r\n\r\n# Atlanta Events October 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices. Experience the diverse and vibrant culture of Atlanta this October!\r\n\r\nStart DateOct 3, 2024ResetCancelApplyEnd DateOct 5, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 508* Oct 03 2024 7:00 PM Thu The Slaps Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $29 Avg. price ~ $33 16 tickets remaining! Tickets\r\n* Oct 03 2024 7:00 PM Thu Sepultura Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $50 Avg. price ~ $85 Available Tickets: 53 Tickets\r\n* Oct 03 2024 7:00 PM Thu La Luz Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $31 Avg. price ~ $39 49 tickets remaining! Tickets\r\n* Oct 03 2024 7:30 PM Thu The Marley Brothers Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $33 Avg. price ~ $162 Available Tickets: 1030 Tickets\r\n* Oct 03 2024 7:30 PM Thu Charli XCX \u0026 Troye Sivan State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $93 Avg. price ~ $263 Available Tickets: 432 Tickets\r\n* Oct 03 2024 7:30 PM Thu The Lion King Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $68 Avg. price ~ $138 Available Tickets: 274 Tickets\r\n* Oct 03 2024 8:00 PM Thu Johnny Marr The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $73 Avg. price ~ $136 19 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Will Burkart City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $49 Avg. price ~ $79 30 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Horse Jumper of Love Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $31 Avg. price ~ $56 16 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu MJ Lenderman and The Wind Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $29 Avg. price ~ $37 19 tickets remaining! Tickets\r\n* Oct 03 2024 8:00 PM Thu Elle King The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $21 Avg. price ~ $86 Available Tickets: 429 Tickets\r\n* Oct 03 2024 8:00 PM Thu vaultboy Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $18 Avg. price ~ $29 18 tickets remaining! Tickets\r\n* Oct 03 2024 8:15 PM Thu Atlanta Falcons vs. Tampa Bay Buccaneers Mercedes-Benz Stadium | Capacity: 83000 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Prices from $42 Avg. price ~ $160 Available Tickets: 2033 Tickets\r\n* Oct 03 2024 8:30 PM Thu Fozzy Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Prices from $85 Avg. price ~ $124 10 tickets remaining! Tickets\r\n* Oct 04 2024 6:00 PM Fri Demon Hunter Heaven Stage at Masquerade - GA | Capacity: 1000 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $42 Avg. price ~ $73 Available Tickets: 55 Tickets\r\n* Oct 04 2024 7:00 PM Fri Kennesaw State Owls vs. Jacksonville State Gamecocks Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $22 Avg. price ~ $78 50 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Rehab Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Prices from $92 Avg. price ~ $107 2 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Sprints Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $36 Avg. price ~ $41 22 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Arrested Development The Ville 30214, 301 Lafayette Ave., Fayetteville, GA, US Prices from $59 Avg. price ~ $69 12 tickets remaining! Tickets\r\n* Oct 04 2024 7:00 PM Fri Jon Pardi Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $21 Avg. price ~ $107 Available Tickets: 1013 Tickets\r\n* Oct 04 2024 7:30 PM Fri Conan Gray Cadence Bank Amphitheatre at Chastain Park | Capacity: 6000 30327, 4469 Stella Drive Northwest, Atlanta, GA, US Prices from $68 Avg. price ~ $135 Available Tickets: 131 Tickets\r\n* Oct 04 2024 7:30 PM Fri Atlanta Symphony Orchestra: How To Train Your Dragon In Concert Atlanta Symphony Hall | Capacity: 1762 30309, 1280 Peachtree St, Atlanta, GA, US Prices from $129 Avg. price ~ $225 10 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri The Hold Steady Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $61 Avg. price ~ $87 20 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri Anna Akana City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $47 Avg. price ~ $85 26 tickets remaining! Tickets\r\n* Oct 04 2024 7:30 PM Fri Kumail Nanjiani The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $45 Avg. price ~ $95 Available Tickets: 324 Tickets\r\n* Oct 04 2024 8:00 PM Fri Midland - Band Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $36 Avg. price ~ $73 Available Tickets: 67 Tickets\r\n* Oct 04 2024 8:00 PM Fri Sarah Kinsley The Loft - GA 30309, 1374 W. Peachtree Street, Atlanta, GA, US Prices from $35 Avg. price ~ $40 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri Friday Pilots Club Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $26 Avg. price ~ $59 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri MJ Lenderman and The Wind Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $39 Avg. price ~ $52 18 tickets remaining! Tickets\r\n* Oct 04 2024 8:00 PM Fri Marco Antonio Solis State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $120 Avg. price ~ $331 Available Tickets: 777 Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[The Slaps Upcoming Events: 2 Altar at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-slaps/)[Sepultura Upcoming Events: 2 Buckhead Theatre Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/sepultura/)[La Luz Upcoming Events: 2 Purgatory Stage at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/la-luz/)[Charli XCX Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/charli-xcx/)[Troye Sivan Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/troye-sivan/)[The Lion King Upcoming Events: 44 Fabulous Fox Theatre - Atlanta Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-lion-king/)[The Marley Brothers Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-marley-brothers/)[Stephen Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/stephen-marley/)[Ziggy Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/ziggy-marley/)[Will Burkart Upcoming Events: 2 City Winery - Atlanta Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/will-burkart/)[MJ Lenderman Upcoming Events: 4 Terminal West At King Plow Arts Center Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/mj-lenderman/)[Johnny Marr Upcoming Events: 2 The Eastern - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/johnny-marr/)[Elle King Upcoming Events: 2 The Tabernacle - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/elle-king/)[Horse Jumper of Love Upcoming Events: 2 Aisle 5 Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/horse-jumper-of-love/)[vaultboy Upcoming Events: 2 Vinyl - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/vaultboy/)[Atlanta Falcons Upcoming Events: 6 Mercedes-Benz Stadium Oct 3, 2024 8:15 PM](https://atlanta-ga.events/tickets/atlanta-falcons/)[Tampa Bay Buccaneers Upcoming Events: 3 Mercedes-Benz Stadium Oct 3, 2024 8:15 PM](https://atlanta-ga.events/tickets/tampa-bay-buccaneers/)[Fozzy Upcoming Events: 2 Madlife Stage \u0026 Studios Oct 3, 2024 8:30 PM](https://atlanta-ga.events/tickets/fozzy/)[Demon Hunter Upcoming Events: 2 Heaven Stage at Masquerade - GA Oct 4, 2024 6:00 PM](https://atlanta-ga.events/tickets/demon-hunter/)[Jacksonville State Gamecocks Football Upcoming Events: 3 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks-football/)[Rehab Upcoming Events: 2 Madlife Stage \u0026 Studios Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/rehab/)[Arrested Development Upcoming Events: 2 The Ville Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/arrested-development/)[Kennesaw State Owls Upcoming Events: 6 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls/)[Jacksonville State Gamecocks Upcoming Events: 3 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks/)[Jon Pardi Upcoming Events: 2 Ameris Bank Amphitheatre Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/jon-pardi/)[Sprints Upcoming Events: 2 Altar at Masquerade - GA Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/sprints/)[Kennesaw State Owls Football Upcoming Events: 6 Fifth Third Stadium Oct 4, 2024 7:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls-football/)[Anna Akana Upcoming Events: 2 City Winery - Atlanta Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/anna-akana/)[The Hold Steady Upcoming Events: 4 Variety Playhouse Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-hold-steady/)[Kumail Nanjiani Upcoming Events: 2 The Tabernacle - GA Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/kumail-nanjiani/)[Atlanta Symphony Orchestra Upcoming Events: 18 Atlanta Symphony Hall Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[How to Train Your Dragon In Concert Upcoming Events: 4 Atlanta Symphony Hall Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/how-to-train-your-dragon-in-concert/)[Conan Gray Upcoming Events: 2 Cadence Bank Amphitheatre at Chastain Park Oct 4, 2024 7:30 PM](https://atlanta-ga.events/tickets/conan-gray/)[Friday Pilots Club Upcoming Events: 2 Vinyl - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/friday-pilots-club/)[Marco Antonio Solis Upcoming Events: 2 State Farm Arena - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/marco-antonio-solis/)[Sarah Kinsley Upcoming Events: 2 The Loft - GA Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/sarah-kinsley/)[Midland - Band Upcoming Events: 2 Coca-Cola Roxy Oct 4, 2024 8:00 PM](https://atlanta-ga.events/tickets/midland-band/)[Gimme Gimme Disco Upcoming Events: 2 The Recher Theatre Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/gimme-gimme-disco/)[Zeke Beats Upcoming Events: 2 Aisle 5 Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/zeke-beats/)[Cozy Worldwide - 90s/2000s Hip Hop and R\u0026B Party Upcoming Events: 2 Ace of Spades Oct 4, 2024 9:00 PM](https://atlanta-ga.events/tickets/cozy-worldwide-90s-2000s-hip-hop-and-rb-party/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.atlantaonthecheap.com/fall-festivals-atlanta/"}
```
and got this result:
```
"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov. - Atlanta on the Cheap[](https://clicky.com/101416864)\r\n\r\n* Skip to primary navigation\r\n* Skip to secondary navigation\r\n* Skip to main content\r\n* Skip to primary sidebar\r\n* Skip to secondary sidebar\r\n\r\n\r\n\r\n[](http://feeds.feedblitz.com/atlonthecheap\u0026x=1)[](https://app.feedblitz.com/f/?Sub=861848\u0026cids=1)Search this site...Atlanta on the Cheap\r\n\r\nHave fun. Be cheap. LIve the Good Life for Less in Atlanta.\r\n\r\n* Shopping\u0026#038; Daily Deals Groupon Activity Deals Groupon Automotive Deals Groupon Beauty \u0026#038; Spa Deals\r\n* Attractions \u0026#038; Museums\r\n* Entertainment\u0026#038; the Arts\r\n* Food \u0026#038; DrinkDeals Groupon Restaurant Deals Groupon Meal-Prep Deals\r\n* TicketDiscounts\r\n* Outdoor Rec \u0026#038; Sports / Fitness\r\n* Calendar Of Events\r\n* Travel On the Cheap\r\n* Resources \u0026#038; Coping Strategies for COVID-19\r\n* Cheapest Gas Prices\r\n\r\nSearch this site...* Home\r\n* Privacy Policy\r\n* More Cheap Cities\r\n* Submit an Event\r\n* About\r\n* Contact Us\r\n\r\nYou are here:[Home](https://www.atlantaonthecheap.com/)/[Arts \u0026amp; Entertainment](https://www.atlantaonthecheap.com/category/arts-and-entertainment/)/80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov.\u0026larr;[The annual Crabapple Fest in downtown Milton is back this October](https://www.atlantaonthecheap.com/crabapple-fest-milton/)[Scarecrow Harvest: Come see over 150 scarecrows in downtown Alpharetta this fall](https://www.atlantaonthecheap.com/scarecrow-harvest-alpharetta/)\u0026rarr;# 80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in Sept., Oct., and Nov.\r\n\r\n![](https://secure.gravatar.com/avatar/fe99c35123550e6e4fd47ee92160be17?s=32\u0026#038;d=blank\u0026#038;r=g)\r\nPOSTED BYHildaShare this:[](https://www.addtoany.com/add_to/facebook?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/twitter?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/pinterest?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/email?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/tumblr?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/reddit?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/copy_link?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/share)The best fall festivals in Atlanta in 2024 \u0026#8212; including state fairs, county fairs, carnivals, harvest celebrations, arts and music festivals, and more!\u0026nbsp;\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2021/08/Fall-Festivals-GRAPHIC.png)\r\nFall festival season in Atlanta kicks off around Labor Day Weekend, when it still feels like summer, and continues until Thanksgiving \u0026#8212; and then it\u0026#8217;s time for tree and menorah lightings, ice rinks, and holiday fun.\r\n\r\nThis list includes festivals, carnivals, and fairs that are taking place in September, October, and November in 2024. This is the most complete listing you will find!\r\n\r\nThe events take place across the extended metro Atlanta region \u0026#8212; so you can plan a day trip or stick close to home.\r\n\r\nAs always, we emphasize festivals with FREE admission, but there are exceptions. The state and county fairs have an admission charge at the gate, and of course you\u0026#8217;ll pay for rides \u0026#8212; but for each one, we keep a list of ticket discounts and ways to save money.\r\n\r\nNOTE: We are still updating details for 2024, but much of the info is current already!\r\n\r\nWe also have a guide to Atlanta summer festivals and a humongous list of Atlanta spring festivals.\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in September\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nFrom Labor Day Weekend\u0026#8217;s end-of-summer vibe through the first cooler days of autumn, you\u0026#8217;ll find plenty to do.\r\n\r\nJump to October FestivalsJump to November Festivals\r\n\r\n## \u2666 Dragon Con Parade\r\n\r\nAugust 31, 2024Downtown Atlanta\r\n\r\nThis annual Labor Day Weekend tradition is more of a pop-culture convention than a traditional festival. The parade draws big crowds; it\u0026#8217;s some of the very best people-watching you\u0026#8217;ll find in downtown ATL!Details for the parade\r\n\r\n![Atlanta, GA, USA - September 5, 2015:  Science fiction creatures scare spectators at the annual Dragon Con Parade on Peachtree Street in Atlanta, GA.](https://www.atlantaonthecheap.com/wp-content/uploads/2019/08/dragon-con-stock-photo-54238809_s-e1567117522154.jpg)\r\nDragon Con 2015. Photo: 123RF## \u2666 Atlanta Black Pride Festival\r\n\r\nAug. 27th thru Sept. 2ndMultiple locations\r\n\r\nFREE. This outdoor festival celebrates diversity with live music, food, art, and more. There are also ticketed events related to Black Pride Weekend.Visit the Black Pride website\u0026nbsp;\r\n\r\n## \u2666 Pioneer Days Festival\r\n\r\nAugust 30 to September 2, 2024Sam Smith ParkCartersville\r\n\r\nThis annual event in Bartow County features carnival rides, games, food, and more. It is a Labor Day Weekend tradition, and takes place at Sam Smith Park.Details for Pioneer Days\r\n\r\n## \u2666 Atlanta Hip Hop Day Festival\r\n\r\nAugust 31 to September 1, 2024Historic 4th Ward Park\r\n\r\nThe FREE music festival will celebrate hip hop culture with entertainment, arts \u0026amp; crafts vendors, and food trucks.Details for Hip Hop Day\r\n\r\n## \u2666 Art in the Park on the Marietta Square\r\n\r\nAugust 31 to September 2, 2024Marietta Square\r\n\r\nThis is the 37th edition of the iconic Labor Day weekend event. Shop for fine art, listen to music, and visit the Chalk Art Spot. Free and family-friendly.Details for Art in the Park\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/marietta-art-in-the-park-610x226.jpg)\r\n## \u2666 House in the Park Music Festival\r\n\r\nSeptember 1, 2024Grant Park\r\n\r\nThis one day music festival is super-affordable, with general admission tickets just $15 when you purchase in advance online. Spend Labor Day weekend in Grant Park.Details for House in the Park\r\n\r\n## Noshfest\r\n\r\nNo info for 2024\r\n\r\nBack for its tenth year in 2022, Noshfest is a celebration of Jewish food and culture. The festival is open to the entire community, and everyone is welcome. A Labor Day Weekend tradition.Details for Noshfest\r\n\r\n## \u2666 Yellow Daisy Festival\r\n\r\nSeptember 5 to 8, 2024Stone Mountain ParkFREE with paid parking\r\n\r\nThis extensive arts-and-crafts festival features more than 300 vendors. You\u0026#8217;ll also find craft demonstrations, a beer garden, festival food, and a Vintage Village for antiques and collectibles. Note that while admission is free, there is a $20 parking fee at Stone Mountain.Go to the website\r\n\r\n## \u2666 Reynoldstown Wheelbarrow Festival\r\n\r\nSeptember 7, 2024Lang Carson ParkAtlanta\r\n\r\nThis fun community festival features an arts and crafts market, live music, a kids zone, and a cornhole tournament. You\u0026#8217;ll find food and drink vendors on site as well. Located within the charming Reynoldstown neighborhood, just a 5-minute walk from the MARTA Station. The festival is FREE to attend. Details for the Wheelbarrow Festival\r\n\r\n## \u2666 Taste of Smyrna\r\n\r\nSeptember 7, 2024Smyrna Village Green\r\n\r\nCome to sample small tastes from 30 local restaurants. There is also a Beer \u0026amp; Wine Garden, a Kids Zone, and live music on stage.Details for Taste of Smyrna\r\n\r\n## \u2666 Canton Multicultural Festival\r\n\r\nSeptember 7, 2024Harmon Park\r\n\r\nThe 4th annual Multicultural Festival moves to a new location, Harmon Park, in 2024. Come to taste traditional foods and enjoy traditional peformances.Details for the Multicultural Festival\r\n\r\n## \u2666 Gwinnett County Fair\r\n\r\nSeptember 12 to 22, 2024Gwinnett County FairgroundsLawrenceville\r\n\r\nThis annual fair features carnival rides, food, entertainment, garden and livestock exhibits, and more. You\u0026#8217;ll pay for both admission and rides, so it makes sense to use the advance purchase discount and buy online.Details \u0026amp; discounts for the Gwinnett Fair\r\n\r\n## \u2666 Coweta County Fair\r\n\r\nSeptember 12 to 22, 2024Coweta County FairgroundsNewnan\r\n\r\nThis annual fair features carnival rides, food, exhibits, shows, and more. There are daily deals that include FREE admission days and discounts on unlimited rides. There is an advance purchase discount on the fair MEGA Pass.Details \u0026amp; discounts for the Coweta County Fair\r\n\r\n## \u2666 Around the World in the DTL\r\n\r\nSeptember 13 and 14, 2024LawrencevilleFREE\r\n\r\nHead to downtown Lawrenceville for this multi-cultural festival that lets you experience diverse cultures from across the globe \u0026#8212; through food, entertainment, and activities. There is a concert with international food trucks on Friday night, with the main festival on Saturday.Details for Around the World in the DTL\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2022/09/around-the-world-in-the-dtl-e1662585030955.jpg)\r\n## \u0026nbsp;\u2666 West Ponce Music Stroll\r\n\r\nSeptember 14, 2024Downtown DecaturFREE\r\n\r\nHead to downtown Decatur for a day of live music in the patios and parking lots of businesses along West Ponce de Leon Avenue. You\u0026#8217;ll also find sidewalk chart artists, vendor booths, and more.Details for the Music Stroll\r\n\r\n## \u2666 Pup-a-Palooza\r\n\r\nSeptember 14, 2024Newtown ParkJohns Creek\r\n\r\nBring your canine companion to the amphitheater at the park for this fun event. Featuring food, fun, doggy costume contests, and a chance to show off your pup\u0026#8217;s best trick.Details for Pup-a-Palooza\r\n\r\n## \u2666 Taste of Snellville\r\n\r\nSeptember 14, 2024Snellville Towne Green\r\n\r\nAn evening of food, fun, and plenty for the kiddos \u0026#8212; a purchased armband gets them unlimited time on the train ride, rock wall, and bounce houses. Food tasting tickets are $1 each. Free admission to this event.Details for Taste of Snellville\r\n\r\n## \u2666 Sandy Springs Fall Arts Festival\r\n\r\nSeptember 14th \u0026amp; 15th, 20246100 Lake Forrest Dr.FREE\r\n\r\nThis is a new fall edition of the spring Artsapalooza festival. There\u0026#8217;s an emphasis on local and regional arts and crafts, along with gourmet food trucks, live music, interactive art stations, and activities for kids.Details for the Fall Arts Festival\r\n\r\n## \u2666 Stone Mountain\u0026#8217;s Pumpkin Festival\r\n\r\nSeptember 14th to November 3, 2024Stone Mountain Park\r\n\r\nThe theme is \u0026#8220;Play by Day; Glow by Night\u0026#8221; and we recommend being there at dusk for the light-up attractions. Walk thru storybook scenes and enjoy a special Pumpkin LaserShow with fireworks. Discounts available!Details \u0026amp; discounts for Pumpkin Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2020/09/pumpkin-festival-stone-mountain-e1601224154504.jpg)\r\nImage courtesy of Stone Mountain Park## \u2666 Alive in Roswell \u0026#8211; September edition\r\n\r\nSeptember 19, 2024(repeats on October 17th)Historic Downtown Roswell\r\n\r\nThis monthly street festival takes place on third Thursdays, from April through October. Live music, food trucks, vendors, kid activities, and more \u0026#8212; spread out along the historic square area.Details for Alive in Roswell\r\n\r\n## \u2666 North Georgia State Fair\r\n\r\nSeptember 19th to 29th, 2024Jim Miller ParkMarietta\r\n\r\nCarnival rides, festival food, live entertainment, and more!Check out our guide to all the FREE admission days \u0026#8212; for teachers, college students, seniors, first responders, etc. \u0026#8212; and the best deals on unlimited ride wristbands.The Fair is conveniently located in Marietta.Details \u0026amp; discounts for the fair\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2021/04/NGSF-2021-photo-9-1024x683.jpg)\r\nPhoto: N. GA State Fair## \u2666 Atlanta Greek Festival\r\n\r\nSeptember 19 to 22, 2024Greek Orthodox Cathedral of the AnunciationAtlanta\r\n\r\nThis festival is back in its traditional format for 2024! Enjoy delicious Greek cuisine to eat on-site or take home, plus traditional Greek music and dance. You can also tour the majestic Greek Orthodox cathedral and shop for Greek-inspired merchandise. Details \u002B discounts for the Greek Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/greek-festival-2024-557x360.jpg)\r\n## \u25CA Adair Park Porches \u0026amp; Pies Festival\r\n\r\nSeptember 21, 2024Adair ParkAtlanta\r\n\r\nA charming neighborhood festival with plenty of pie \u0026#8212; as well as a beer garden, artist market, kid activities and more. Admission is free, but a pie-tasting pass costs $10.Go to the website\r\n\r\n## \u2666 Urban Regatta\r\n\r\nSeptember 21, 2024Castleberry HillFree to attend\r\n\r\nThe Urban Regatta celebrates the artistic culture in Atlanta\u0026#8217;s Castleberry Hill neighborhood. It includes an artist market, music, pop-up galleries, and the main event \u0026#8212; a regatta-style race using bicycles instead of boats. Details for Urban Regatta\r\n\r\n## \u2666 Chamblee Chalk Walk \u0026amp; Artist Market\r\n\r\nSeptember 21, 2024Chamblee City HallFREE\r\n\r\nCome to see the sidewalks of downtown Chamblee transformed with amazing chalk art. You can try your own hand in the Community Chalk Spot, browse the artist market, eat and drink in the open container district, and have a day of fun.Details for Chalk Walk\r\n\r\n## \u2666 Buckhead Fine Arts Festival\r\n\r\nSeptember 21 and 22, 2024Buckhead Village DistrictFREE\r\n\r\nThis is a two-day arts festival, featuring 100 vendor booths, live music, artist demonstrations, food trucks, and more.Details for the Fine Arts Festival\r\n\r\n## \u2666 Suwanee Fest\r\n\r\nSeptember 21 and 22, 2024Town Center ParkSuwaneeFREE\r\n\r\nThis annual event kicks off with a parade on Saturday morning. There is also a juried art market, kids fun zone, plenty of food, and live entertainment all through the day. Free shuttle from designated parking areas.Details for Suwanee Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/suwanee-fest-no-year-e1722286789994-481x360.png)\r\n## \u2666 Shakerag Arts \u0026amp; Crafts Festival\r\n\r\nSeptember 21 and 22, 2024McIntosh Trail Recreation ComplexPeachtree CityFREE\r\n\r\nAn annual event in Peachtree City. This outdoor crafts fair takes place among the pines at Shakerag Knoll. Vendors, food, activities, live music, and more.Details for Shakerag\r\n\r\n## \u2666 Japan Fest\r\n\r\nSeptember 21 and 22, 2024Gas South Convention CenterDuluth\r\n\r\nExperience Japanese culture, food, music, dance, anime, traditions, martial arts and more at this huge exhibition. There are multiple entertainment stages and a dedicated food court. Advance purchase tickets are $16, and you\u0026#8217;ll pay $20 at the door.Details for Japan Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2016/06/japanfest-atl-2017-1024x575.png)\r\n## \u2666 Cumming Art Fest\r\n\r\nSeptember 21 and 22, 2024Cumming City CenterFREE\r\n\r\nThis event is returning for its third year. It is FREE to attend and takes place at the new Cumming City Center complex. You\u0026#8217;ll find art vendors, a Kidz Zone, festival food, and more.Details for Cumming Art Fest\r\n\r\n## \u2666 Roswell Fall Arts Festival\r\n\r\nSeptember 21 and 22, 2024Grounds of Roswell City HallFREE\r\n\r\nFun in historic downtown Roswell! Come for shopping, live music, food trucks, kids\u0026#8217; activities, and plenty of small town charm. Note the change in location, due to construction on the Town Square.Details for Roswell Arts Festival\r\n\r\n## \u2666 Peachtree Corners Festival\r\n\r\nSeptember 21 and 22, 2024The Peachtree Corners Town GreenFREE\r\n\r\nThe 13th annual edition of this event promises two days of fun.\u0026nbsp;You\u0026#8217;ll find live music, vendor booths, car show, and inflatables for the kids.A special Friday night concert kicks off the event. Free parking.Details for the festival\r\n\r\n## \u2666 Taste of Canton\r\n\r\nSeptember 26, 2024The Mill at Etowah FREE admission\r\n\r\nThis annual tasting event allows you to try out the local restaurants through small samples \u0026#8212; including tastes of appetizers, desserts, and entrees. Admission is FREE and tasting tickets are $1 \u0026#8212; note that some items require more than one ticket. There is usually a cash bar on site.Details for Taste of Canton\r\n\r\n## \u2666 Georgia State Fair\r\n\r\nSeptember 27th to October 6, 2024Atlanta Motor SpeedwayHampton, GA\r\n\r\nCome for the carnival rides, food, free concerts, exhibits, and more. The fair is here for about ten days, so you\u0026#8217;ll have two weekend options. There are daily deals on admission and unlimited ride wristbands \u0026#8212; so click through to see our guide to the best value days.Details \u0026amp; discounts for the State Fair\r\n\r\n## \u2666 Blue Stone Arts \u0026amp; Music Festival\r\n\r\nSeptember 27th \u0026amp; 28th, 2024City Springs districtSandy SpringsFREE\r\n\r\nThis is a new festival, featuring an artist market and live music on two stages. There is an extensive concert line-up both on Friday night, and all day Saturday. Plan to park off-site and use the FREE shuttle.Details for Blue Stone Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/09/Blue-Stone-Arts-and-Music-Festival-graphic-no-year-1-610x343.jpg)\r\n## \u2666 East Atlanta Strut\r\n\r\nSeptember 28, 2024East Atlanta VillageFREE\r\n\r\nThis iconic street festival celebrates East Atlanta\u0026#8217;s culture, art, and quirkiness! This is a \u0026#8220;porchfest\u0026#8221; style party that stretches throughout the neighborhood.Details for the Strut\r\n\r\n## \u2666 Collard Greens Cultural Festival\r\n\r\nSeptember 29, 2024Lake Sheryl 4540 Lee Rd SWSnellville, GA 30039\r\n\r\nCome for soul food, music genres from jazz to reggae, local vendors and more. Save room for the collard green ice cream! Visit the festival website\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/collard-greens-cultural-festival-e1722343884538-591x360.png)\r\n## \u2666 Alpharetta Art in the Park\r\n\r\nSeptember 28th and 29th, 2024(repeats again in October)Brooke Street ParkAlpharettaFREE\r\n\r\nThis art market places an emphasis on handcrafted goods by local artists and craftspeople. It occurs on the fourth weekend of the month, June thru September.Details for Art in the Park\r\n\r\n## \u2666 Riverfest\r\n\r\nSeptember 28th \u0026amp; 29th, 2024Etowah River ParkCanton\r\n\r\nThis is a fundraiser, so you\u0026#8217;ll pay a $7 admission fee (kids 10 and under are FREE). Come to shop for arts and crafts, enjoy local musicians and performers, eat great food, and browse the scarecrow display.Details for Riverfest\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nSeptember 15, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## \u2666 Sweet Auburn Music Fest\r\n\r\nSeptember 28th and 29th, 2024Sweet Auburn DistrictDowntown AtlantaFREE\r\n\r\nCome the historic Sweet Auburn district for a street festival featuring R\u0026amp;B, hip-hop, gospel music, and more. You\u0026#8217;ll also find great food, a children\u0026#8217;s area, vendors, and more.Details for Sweet Auburn Fest\r\n\r\n## \u2666 Duluth Fall Festival\r\n\r\nSeptember 28th and 29th, 2024Duluth Town GreenFREE\r\n\r\nThis is the 37th year for this beloved fall festival. Two full days of arts/crafts vendors, food trucks, music and dance, a 5K, and a parade. You\u0026#8217;ll find non-stop entertainment on two stages.Details for Duluth Fall Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2017/09/duluth-fall-festival-logo-e1568676416984.jpg)\r\n## \u2666 Sunday in the Park \u0026amp; Tunes from the Tombs\r\n\r\nSeptember 29, 2024Oakland CemeteryAtlanta\r\n\r\nOakland Cemetery\u0026#8217;s beloved fall festival is based on the Victorian tradition of spending a day of leisure in the cemetery. It has always been a free event \u0026#8212; however, in 2023, it\u0026#8217;s being combined with Tunes from the Tombs and is a ticketed event.Details for this event\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/tunes-from-the-tombs-no-year-610x320.jpg)\r\n## \u2666 Serb Fest\r\n\r\nSeptember 28th and 29th, 2024Sts. Peter \u0026amp; Paul Serbian Orthodox ChurchLilburn\r\n\r\nA cultural experience like no other in Atlanta! You will experience first-hand Serbian hospitality, culture, history, live entertainment, colorful clothing, and mouth-watering food.Details for Serb Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in October\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nThe cooler autumn air has finally arrived and it\u0026#8217;s a great time to be outdoors! You\u0026#8217;ll find art markets, carnival rides, harvest celebrations, Oktoberfests, spooky fun, and more.\r\n\r\nJump to September FestivalsJump to November Festivals\r\n\r\n## \u2666 Decatur Wine Festival: Early Bird Ticket Sales\r\n\r\nThe festival is on November 9, 2024 \u0026#8212; but early bird ticket sales close in late October, so this is a reminder to buy early and save $12 in ticket costs and fees!Details for the Wine Festival\r\n\r\n## \u2666 Georgia State Fair\r\n\r\nThrough October 6th, 2024Atlanta Motor SpeedwayHampton, GA\r\n\r\nCome for the carnival rides, food, free concerts, exhibits, and more. The fair is here for ten days, which includes two weekends. There are daily deals on admission and unlimited ride wristbands \u0026#8212; so be sure to click through to see our guide on the best value days.Details \u0026amp; discounts for the State Fair\r\n\r\n## \u2666 Cumming Country Fair \u0026amp; Festival\r\n\r\nOctober 3 to 13, 2024Cumming Fairgrounds\r\n\r\nCarnival rides, the lights of the midway, fair food, live concerts, and daily attractions! If you plan to head to the Fairgrounds, check out our list of discounts and ride deals before you go.Details \u0026amp; discounts for the Cumming Fair\r\n\r\n## \u2666 Decatur Book Festival\r\n\r\nOctober 4 and 5, 2024Downtown Decatur\r\n\r\nThe annual book festival has scaled down immensely \u0026#8212; and transitioned into a one-day format, with a ticketed keynote event the night before. It\u0026#8217;s now held on the grounds of a local church, with author sessions inside and vendor tents on the grounds.Details for the Book Festival\r\n\r\n## \u2666 Atlanta Fair\r\n\r\nOctober 4 to November 3, 2024Old Turner FieldAtlanta\r\n\r\nThis is the region\u0026#8217;s only urban fair, taking place at Turner Field in downtown Atlanta. There\u0026#8217;s a carnival midway with rides, plenty of food, nightly entertainment, and more. It will be here for all of October.Details for Atlanta Fair\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/03/Atlanta-Fair-graphic-1024x590.jpg)\r\nImage: Atlanta Fair## \u2666 Pumpkin Festival at Stone Mountain\r\n\r\nAll month, through November 3, 2024Stone Mountain Park\r\n\r\nThe theme is \u0026#8220;Play by Day; Glow by Night\u0026#8221; and we recommend being there at dusk for the light-up attractions. Walk thru storybook scenes and enjoy a special Pumpkin LaserShow with fireworks. Discounts available!Deals \u0026amp; discounts for Pumpkin Festival\r\n\r\n## \u25CA Atlanta Korean Festival\r\n\r\nOctober 5, 2024Korean Community CenterNorcross\r\n\r\nA celebration of Korean and Korean-American culture and heritage! You\u0026#8217;ll find traditional music and cultural performances, martial arts, plenty of food, and family-friendly activities. There is a $10 admission fee for adults, and $5 for kids.Go to the website\r\n\r\n## \u2666 Milton Crabapple Fest\r\n\r\nOctober 5, 2024Downtown MiltonFREE\r\n\r\nMilton\u0026#8217;s annual fall festival takes place the first weekend in October along Crabapple Road. This is an arts and crafts festival, with approximately 100 vendor booths, food, and family activities.Details for Crabapple Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/07/crapabble-fest-milton-no-year-604x360.jpg)\r\n## \u2666 Taste of Tucker\r\n\r\nOctober 5, 2024Main StreetDowntown Tucker\r\n\r\nCome hungry and try out a wide variety of tasting samples, including appetizers, entrees, and desserts. There are also art vendors, a beer garden, and a Kids Zone for the young ones.Details for Taste of Tucker\r\n\r\n## \u2666 Suwanee Taste of BBQ \u0026amp; Music Festival\r\n\r\nOctober 5, 2024Town Center ParkSuwaneeFREE admission\r\n\r\nThis all-day music festival takes place in downtown Suwanee. Admission, parking, and entertainment is free. The BBQ tasting tickets s are $1 each, and each taste requires one to four tickets. You can also buy a full plated barbecue meal. There is also a kids fun area and an artist market.Details for this event\r\n\r\n## \u2666 Candler Park Fall Fest\r\n\r\nOctober 5 and 6, 2024Candler ParkAtlantaFREE\r\n\r\nThis popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more.Details for Fall Fest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2018/09/candler-park-street-signs-e1569941757312.jpg)\r\n## \u2666 Norcross Art Splash\r\n\r\nOctober 5 and 6, 2024Lillian Webb ParkDowntown NorcrossFREE\r\n\r\nThis annual fall festival features 90\u002B arts and crafts vendors, plenty of food, and an interactive Kid Zone with face painting, inflatables, and more. Located in the historic downtown area.Details for Art Splash\r\n\r\n## Prost on Perry\r\n\r\nNo info for 2024\r\n\r\nThis traditional Oktoberfest celebration takes place in downtown Lawrenceville, on the town square. Come for beer, food, live entertainment, and a good time. Family-friendly.Details for Prost on Perry\r\n\r\n## \u2666 Fall Festival on Ponce\r\n\r\nOctober 5 and 6, 2024Olmsted Linear ParksAtlantaFREE\r\n\r\nThis intown festival features vendor booths of arts and crafts, a children\u0026#8217;s play area, local food and beverage vendors, and live acoustic music. It is spread across three of the Olmsted Linear Parks.Details for Festival on Ponce\r\n\r\n## \u25CA Conyers Latin Festival\r\n\r\nOctober 6, 2024Olde Town Conyers FREE\r\n\r\nThis annual celebration observes the history, culture, and contributions of those whose ancestors came from Spain, Mexico, the Caribbean, and Central and South America. You\u0026#8217;ll find food, an arts \u0026amp; crafts marketplace, live entertainment, a Kid Zone, and more. Visit the festival website\r\n\r\n## \u2666 Wire \u0026amp; Wood Music Festival\r\n\r\nOctober 11 and 12, 2024Downtown AlpharettaFREE\r\n\r\nThis FREE music fest features more than 30 performances by singer-songwriters on six outdoor stages located throughout the Downtown Alpharetta area. An Alpharetta cultural tradition since 2013.Details for Wire \u0026amp; Wood\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/wire-and-wood-2024-610x226.png)\r\n## \u2666 Taste of Acworth\r\n\r\nOctober 12, 2024Downtown Acworth\r\n\r\nThis affordable food festival features FREE admission, and small bites for just $1 to $5 each. It takes place along historic Main Street and the surrounding area.Details for Taste of Acworth\r\n\r\n## \u2666 Lilburn Daze\r\n\r\nOctober 12, 2024Lilburn City ParkFREE\r\n\r\nThis beloved fall arts \u0026amp; crafts is in its 50th year! It is extremely family-friendly, with plenty of activities for the kids. You\u0026#8217;ll also find food, vendor booths, and live entertainment all day long. Plan to use the free shuttle.Details for Lilburn Daze\r\n\r\n## \u2666 The Great Pumpkin Fest\r\n\r\nOctober 12, 2024Cherokee Veterans ParkCantonFREE\r\n\r\nThis annual fall and Halloween festival takes place in Cherokee County. You\u0026#8217;ll find craft vendors, food, a DJ, hay rides, games, bounce houses for the kids, trick-or-treating, and more.Details for Pumpkin Fest\r\n\r\n## \u2666 Old Fourth Ward Park Arts Festival\r\n\r\nOctober 12 and 13, 2024Old 4th Ward ParkFREE\r\n\r\nThis two-day arts festival features live acoustic music, about 150 vendor booths, food, and fall fun \u0026#8212; located at historic Old 4th Ward Park.Details for the O4W Arts Festival\r\n\r\n## \u2666 Chalktoberfest\r\n\r\nOctober 12 and 13, 2024Marietta SquareFREE\r\n\r\nIt\u0026#8217;s a festival that celebrates street art and craft beer! Enjoy a fall afternoon at Chalktoberfest, in the Marietta Square. Enjoy live music and eat great food, while you watch the work of chalk artists unfold in front of your eyes. FREE to attend, but the craft beer tasting is a ticketed event.Details for Chalktoberfest\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2016/10/chalktoberfest-from-stock-37714032_s-e1570630137637.jpg)\r\nMarietta Chalktoberfest 2014 \u0026#8211; Image: 123RF## Atlanta Pride Festival \u0026amp; Parade\r\n\r\nOctober 12 and 13, 2024Piedmont Park\r\n\r\nAn annual October event! Atlanta Pride features a festival and parade in Midtown, plus a wide variety of associated events around town. Details for Atlanta Pride\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2018/10/pride-hands-forming-heart-43060357_s-e1570548745417.jpg)\r\nDeposit Photos## \u2666 Georgia Apple Festival\r\n\r\nOctober 12 and 13, 2024October 19 and 20, 2024Ellijay, GA\r\n\r\nThis is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!Details for the Apple Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/Apple-picking-114688623_s-e1570031034124.jpg)\r\nDeposit Photos## \u2666 Alive in Roswell Street Festival\r\n\r\nOctober 17, 2024Historic Downtown RoswellFREE\r\n\r\nThis monthly street festival takes place on third Thursdays, from April through October. Live music, food trucks, vendors, kid activities, and more \u0026#8212; spread out along the historic square area.Details for Alive in Roswell\r\n\r\n## \u2666 Halloween \u0026amp; Spooky Fun\r\n\r\n\u0026#8216;Tis the season to be spooky! Check out our huge list of celebrations for Halloween and Dia de los Muertos \u0026#8212; including festivals, ghost tours, trick-or-treat events, and more. The events start in mid-October and continue through October 31st. View our huge list of Halloween events\r\n\r\n![20948527 - monster movies with a zombie hand holding a cinema or theater ticket stub as a creepy halloween or scary entertainment symbol ](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/halloween-green-hand-holding-ticket-20948527_s-e1570221116120.jpg)\r\nDeposit Photos## \u2666 Cumming Greek Festival\r\n\r\nOctober 18 to 20, 2024Saints Raphael and Nicholas Greek Orthodox ChurchCumming\r\n\r\nBe Greek for a day! Come out for authentic Greek cuisine, a vendor market, and continuous live entertainment and cultural performances. You\u0026#8217;ll also have a chance to tour the Greek Orthodox church.Details for the Greek Festival\r\n\r\n## \u2666 Fall-0-Ween Fest\r\n\r\nOctober 19, 2024Swift-Cantrell ParkKennesawFREE\r\n\r\nThis event includes a candy trail, costume contests, scavenger hunts, games, crafts, music, fooc, and more. It\u0026#8217;s followed by a Halloween movie at sunset.Details for Fall-O-Ween\r\n\r\n## Oakhurst Porchfest\r\n\r\nOctober 19, 2024Oakhurst NeighborhoodAtlantaFREE\r\n\r\nThis grassroots music festival turns front porches into stages and yards into venues. You can wander at leisure through one square mile of the Oakhurst neighborhood, taking in mini-concerts. In past years, the event has featured over 200 performances.Details for Porchfest\r\n\r\n## Marietta HarvestFest\r\n\r\nOctober 19, 2024Marietta SquareFREE\r\n\r\nThis annual October event includes an arts/crafts market, scarecrow display, touch-a-truck event, and a special Halloween Happenings festival for kids 12 and under \u0026#8212; complete with trick-or-treating and a costume contest. In Glover Park on the Marietta Square.Details for Marietta HarvestFest\r\n\r\n## \u2666 Snellville Fall Festival\r\n\r\nOctober 19, 2024Snellville Town GreenFREE\r\n\r\nFeaturing a Halloween costume contest for kids, adults, and dogs! You\u0026#8217;ll also find vendor booths, food, a concert, and plenty of activities for kids (an activity wristband is $15).Details for the Fall Festival\r\n\r\n## Red Oak Art Festival\r\n\r\nNo info for 2024 yet \u0026#8212; stay tuned!New Life AME Zion ChurchCity of South FultonFREE\r\n\r\nThis FREE and family-friendly event is all about the arts! It features interactive, community art projects, mini theater workshops, an instrument petting zoo, local food trucks, and more.Details for the festival\r\n\r\n## \u2666 Loganville Autumn Fest\r\n\r\nOctober 19, 2024Loganville Town GreenFREE\r\n\r\nA classic fall festival on the Loganville Towne Green. You\u0026#8217;ll find arts vendors, food, live entertainment, and plenty of stuff to do for the kiddos.Details for Autumn Fest\r\n\r\n## Woodstock International Food Festival\r\n\r\nNo 2024 dates yet \u0026#8212; stay tuned!St. Elizabeth Orthodox Church\r\n\r\nThis festival began in 2018 as a way to honor the diversity of the community. Besides ethnic cuisines from all corners of the world, there will be live music, dancing, church tours, entertainment, and children\u0026#8217;s activities.Details for the Food Festival\r\n\r\n## Spirits \u0026amp; Spice Festival\r\n\r\nOctober 19, 2024Downtown NewnanFREE\r\n\r\nEat, drink, and be spooky at this one-day street festival in charming downtown Newnan. Enjoy a chili cook-off and themed cocktails, plus live music, and activities for the kids.Details for Spirits \u0026amp; Spice\r\n\r\n## Olde Town Fall Festival\r\n\r\nOctober 19, 2024Olde Town ConyersFREE\r\n\r\nThis annual event in Conyers features a full day of arts and craft vendors, festival food, live entertainment, and kid\u0026#8217;s activities. You\u0026#8217;re invited to come prepared to enter a costume contest!Details for Fall Festival\r\n\r\n## Brookhaven International Festival\r\n\r\nOctober 19 and 20, 2024Peachtree Creek GreenwayFREE\r\n\r\nExpanded to two days in 2024! This event features live multi-cultural entertainment, international cuisine, a Kids Zone, interactive art stations, and more.Details for Brookhaven International Festival\r\n\r\n## Alpharetta PopFest\r\n\r\nNo 2024 date yet \u0026#8212; stay tuned!Brooke Street ParkAlpharetta\r\n\r\nA one-day outdoor music festival in support of Alzheimer research. Come for food, drink, and live music.Details for PopFest\r\n\r\n## Fall Festival in McDonough\r\n\r\nNo 2024 date yet \u0026#8212; stay tuned!87 Griffin St(green space across from the church)\r\n\r\nThis is a sensory-friendly fall festival for the community \u0026#8212; featuring face painting, carnival games, trunk-or-treating, and more. Hosted by First Baptist McDonough.Details for Fall Festival\r\n\r\n## Johns Creek Arts Festival\r\n\r\nOctober 19 and 20, 2024Atlanta Athletic ClubJohns CreekFREE\r\n\r\nThis art market features over 100 arts vendors from around the country, so come ready to shop. You\u0026#8217;ll also find festival food, a ticketed Kid Zone, and continuous live concerts by local musical performers.Details for this event\r\n\r\n## Romanian Festival\r\n\r\nNo 2024 dates yet \u0026#8212; stay tuned!Saints Constantine and Elena Romanian Orthodox ChurchLilburn\r\n\r\nCelebrate Romanian culture with delicious food, traditional music and dance, and more.Details for Romanian Festival\r\n\r\n## The Highland Games \u0026amp; Scottish Festival\r\n\r\nOctober 19 and 20, 2024Stone Mountain Park\r\n\r\nStone Mountain plays host to pipers, drummers, dancers, Scottish athletic competitions, traditional food, clan tents, vendors selling imported items, and more. A fun and educational cultural experience! This is a ticketed event. The gate price is $25 and there is a small discount for buying online in advance. Go to the festival website\r\n\r\n## \u2666 Hogansville Hummingbird Festival\r\n\r\nOctober 19 and 20, 2024Main StreetHogansvilleFREE\r\n\r\nA charming, small-town street fest in historic downtown Hogansville, which is about 30 minutes south of the ATL Airport. Vendor booths, music, food, fun.Details for Hummingbird Festival\r\n\r\n## \u2666 Magnolia Arts Festival\r\n\r\nOctober 19 and 20, 2024Drake FieldPeachtree CityFREE\r\n\r\nThis annual Peachtree City event includes a juried art sale, student art exhibit, food trucks, kids activities, live entertainment, and plenty of fall fun. This is the 7th year for this fall festival on the south side of town.Details for Magnolia Arts Festival\r\n\r\n## LakeFest\r\n\r\nOctober 19 and 20, 2024Lake Shore DrivePine Lake, GAFREE\r\n\r\nPine Lake may be the smallest city in Dekalb County, but it hosts a huge festival each fall. You\u0026#8217;ll find live music, food, artist booths, and Floatzilla \u0026#8212; a floating parade.Go to the website\r\n\r\n## Little 5 Points Halloween Festival \u0026amp; Parade\r\n\r\nOctober 19 and 20, 2024FREE\r\n\r\nCelebrate the spooky, the quirky, and the bizarre in L5P this Halloween! The parade is back with a new route along Moreland Ave. You\u0026#8217;ll also find vendors, food, booze, live music, Drag Queen Karaoke, and more.Details for L5P Halloween Festival\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nOctober 20, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## \u25CA Decatur BBQ \u0026amp; Bands Festival\r\n\r\nOctober 26, 2024Oakhurst VillageFREE\r\n\r\nLocal BBQ restaurants present a variety of options, including vegan and gluten-free. Deserts, frozen treats, and adult beverages are also available to purchase. Live music from noon to 4 p.m. Admission is FREE; just bring your wallet for food and drink.Go to the website\r\n\r\n## \u25CA Benteen Park Fall Fest\r\n\r\nOctober 26, 2024Benteen Park1410 Casanova St SEAtlanta, GA 30315\r\n\r\nThe east end of Casanova Street will be closed for the festival, which will fill the fields and circle at the end of the road. From 11 a.m. to 6 p.m., you\u0026#8217;ll find amazing food vendors, an awesome music stage, and an artist market. You\u0026#8217;re invited to bring chairs, blankets, etc. to spread out and enjoy the park.Visit the festival website\r\n\r\n## \u2666 Alpharetta Art in the Park\r\n\r\nOctober 26th and 27th, 2024Brooke Street ParkAlpharettaFREE\r\n\r\nThis art market places an emphasis on handcrafted goods by local artists and craftspeople. It occurs on the fourth weekend of the month, June thru September.Details for Art in the Park\r\n\r\n## Smyrna Crafts \u0026amp; Drafts Festival\r\n\r\nOctober 26 and 27, 2024Smyrna Town GreenFREE\r\n\r\nFormerly known as the Fall Jonquil Festival, this newly re-branded fall is in its second year. It features 150\u002B arts and crafts vendors, food, live entertainment, and activities for the kids.Details for Crafts \u0026amp; Drafts\r\n\r\n## \u2666 Johns Creek Diwali Festival\r\n\r\nOctober 26 and 27, 2024Johns Creek City HallFREE\r\n\r\nCelebrate Diwali, the traditional Hindu festival of light! You\u0026#8217;ll find retail and food vendors, live entertainment, and a drone light show. The symbolic meaning of Diwali is the triumph of light over dark, good over evil, and knowledge over ignorance.Details for Diwali Festival\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/diwali-601x360.jpg)\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S.jpg)\r\n\r\n\r\n# Festivals in November\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S.jpg)\r\n\r\n\r\nFestival season is winding down, but there are still plenty of opportunities to enjoy the cool autumn air \u0026#8212; and perhaps get a headstart on holiday gift shopping.\r\n\r\nJump to October FestivalsJump to September Festivals\r\n\r\n## Atlanta Fair\r\n\r\nThrough November 6thOld Turner FieldDowntown Atlanta\r\n\r\nThis is Atlanta\u0026#8217;s only urban fair, taking place in the lots outside the Old Turner Field. There\u0026#8217;s a carnival midway with rides, plenty of food, and more.Details for Atlanta Fair\r\n\r\n## Dia de los Muertos Festival\r\n\r\nNovember 2, 2024Lawrenceville Arts CenterFREE\r\n\r\nHave a cultural experience at this traditional Day of the Dead festival. Enjoy food, traditional music, and ceremonial traditions.Details for Dia de los Muertos\r\n\r\n## The Rescue Dog Games\r\n\r\nNovember 2, 2024\r\n\r\nThis is a fun festival for dogs and their humans in Piedmont Park. You\u0026#8217;ll find interactive games, pet-themed vendors, food trucks, photo booths, and a Howl-o-Ween doggy costume contest.Details for Rescue Dog Games\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2024/08/rescue-dog-games-2024-graphic-610x343.png)\r\n## Taste of Kennesaw\r\n\r\nNovember 2, 2024Kennesaw Depot Park\r\n\r\nThis one-day food festival allows you to sample food from local restaurants. The small plates range from $1 to $4 per taste. There\u0026#8217;s also a kids area with inflatables, a beer garden, and live music on two states.Details for Taste of Kennesaw\r\n\r\n## Chomp \u0026amp; Stomp Chili Cook-off and Bluegrass Festival\r\n\r\nNovember 2, 2024Cabbagetown neighborhoodAtlantaFREE\r\n\r\nThis one-day event is a fall favorite in the charming Cabbagetown neighborhood of Atlanta. You\u0026#8217;ll be able to taste chili samples, shop the artist market, and listen to bluegrass on five stages.Details for Chomp \u0026amp; Stomp\r\n\r\n## Suwanee Wine Festival\r\n\r\nNovember 2, 2024Suwanee Town Center Park\r\n\r\nSip and stroll your way around the park, enjoying samples of 150 wines from all over the world \u0026#8212; plus a Georgia winery section. There is also live music, art vendors, food trucks, a beer tent, and football on a big screen. You can save a few bucks with early-bird tickets, which usually go on sale in late summer.Go to the festival website\r\n\r\n## Chastain Park Fall Arts Festival\r\n\r\nNovember 2 and 3, 2024Chastain ParkAtlantaFREE\r\n\r\nThis is the fall edition of this popular festival, after a successful run in the spring. You can start your holiday shopping at the art market, plus enjoy the children\u0026#8217;s area, local gourmet food and drink, and acoustic musical performances.Details for Fall Arts Festival\r\n\r\n## Dia De Los Muertos \u0026#8212; Day of the Dead Festival in East Point\r\n\r\nNovember 4, 2024 \u0026#8212; not yet confirmed East Point City HallFREE\r\n\r\nA traditional Day of the Dead celebration with food, dance, games, face painting and more.Details for Day of the Dead\r\n\r\n## Lawrenceville Harvest Festival\r\n\r\nNovember 9, 2024The Lawrenceville LawnFREE\r\n\r\nBack for its third year! The fall fun includes hayrides around the Lawrenceville Square, a petting zoo, four live country music acts, food, vendors, and more.Details for Harvest Festival\r\n\r\n## \u2666 Decatur Wine Festival\r\n\r\nNovember 9, 2024Old Courthouse SquareDowntown Decatur\r\n\r\nTickets are not cheap for this fun event, so you\u0026#8217;ll want to take advantage of early-bird pricing, and buy your ticket in advance. Details for the Wine Festival\r\n\r\n## Suwanee Chili Cook-Off and Music Festival\r\n\r\nNovember 9, 2024Suwanee Town CenterFREE\r\n\r\nListen to live music, play in the Kids Zone, shop with arts and crafts vendors, and taste lots of great chili before you vote for your favorite. FREE to attend, but chili sampling requires a $10 wristband.Details for the Cook-Off\r\n\r\n## Stone Mountain Christmas\r\n\r\nNovember 9, 2024 through January 5, 2025Stone Mountain Park\r\n\r\nVisit Stone Mountain for this festival of holiday magic, dazzling lights, a nightly parade, Santa\u0026#8217;s flight through the sky, and spectacular shows. The Christmas Village is open on weekends and during the school holiday break. Click through for available discounts!Details \u0026amp; discounts for Stone Mountain Christmas\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/10/stone-mountain-christmas-large-flyer-1024x1024.jpg)\r\n## \u2666 Atlanta Chili Cook Off\r\n\r\nNovember 16, 2024Brook Run ParkDunwoody\r\n\r\nThis ticketed food festival promises a day of fun. Come to taste chili from 50\u002B amateur cooks, restaurants, and caterers, while listening to a DJ spin your favorite tunes. Details \u0026amp; ticket discounts for Chili Cook Off\r\n\r\n## \u2666 Atlanta Streets Alive\r\n\r\nNovember 17, 2024West End to Grant ParkFREE\r\n\r\nThis is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u0026#8212; so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun. It\u0026#8217;s the ultimate \u0026#8220;open streets\u0026#8221; event in metro Atlanta, and it\u0026#8217;s a heckuva good time.Details for Streets Alive\r\n\r\n## Atlanta Christkindl Market\r\n\r\nNovember 22nd through December 24, 2024Buckhead VillageFREE\r\n\r\nThis German-style street festival and holiday market is one of our favorite events. Enjoy authentic German food and shop your way through the charming vendor booths, styled as Bavarian chalets.Details for Christkindl Market\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2019/11/christkindl-market-1-e1574206875851.jpg)\r\n## Festival of Trees\r\n\r\nNovember 22nd thru December 1st, 2024(closed Thanksgiving Thursday \u002B Friday)Georgia World Congress CenterDowntown Atlanta\r\n\r\nThe Georgia Festival of Trees is a special five-day holiday event for all ages. Wander through a beautiful forest of dazzling holiday trees, shop for unique gift shops, and enjoy treats from local vendors.Details \u0026amp; discounts for Festival of Trees\r\n\r\n## \u2666 HOLIDAY FUN\r\n\r\nIt\u0026#8217;s late November and festival season is over \u0026#8212; but the fun doesn\u0026#8217;t stop here!\r\n\r\nStay with us for tons of holiday fun, including:\r\n\r\n* Spectacular holiday light displays\r\n* FREE photos with Santa (60\u002B opportunities)\r\n* Huge list of seasonal holiday \u0026amp; winter events\r\n* 40 great ways to celebrate the Winter Solstice\r\n\r\nOur big list of holiday fun includes tree \u0026amp; menorah lightings, skating rinks, live nativity scenes, parades, holiday markets, snow play areas, Hanukkah celebrations, Kwanzaa events, and more!FIND YOUR HOLIDAY FUN\r\n\r\n\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-Depositphotos_81466966_S-610x51.jpg)\r\n# Fall Festivals\r\n\r\n![](https://www.atlantaonthecheap.com/wp-content/uploads/2023/07/narrow-fall-banner-autumn-leaves-FLIPPED-Depositphotos_81466966_S-610x51.jpg)\r\n\r\n\r\n\r\n\r\nShare this:[](https://www.addtoany.com/add_to/facebook?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/twitter?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/pinterest?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/email?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/tumblr?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/reddit?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/add_to/copy_link?linkurl=https%3A%2F%2Fwww.atlantaonthecheap.com%2Ffall-festivals-atlanta%2F\u0026amp;linkname=80%2B%20fall%20festivals%20%26%20fairs%20around%20metro%20Atlanta%3A%20Fun%20things%20to%20do%20in%20Sept.%2C%20Oct.%2C%20and%20Nov.)[](https://www.addtoany.com/share)Posted by Hilda on September 30, 2024 | Updated October 2, 2024 Filed Under: Arts \u0026amp; Entertainment\u0026nbsp;\u0026middot; Featured\u0026nbsp;\u0026middot; Festivals Tagged With: Fall Fun\u0026middot; Roundups\r\n\r\n## Reader Interactions\r\n\r\n### Trackbacks\r\n\r\n1. jungle boys weed says: October 2, 2024 at 6:13 pm \u0026#8230; [Trackback] [\u0026#8230;] Read More here to that Topic: atlantaonthecheap.com/fall-festivals-atlanta/ [\u0026#8230;]\r\n2. URL says: October 1, 2024 at 12:50 pm \u0026#8230; [Trackback] [\u0026#8230;] There you will find 25997 more Infos: atlantaonthecheap.com/fall-festivals-atlanta/ [\u0026#8230;]\r\n3. The Best Free Things to Do in Atlanta in 2024 - says: August 20, 2024 at 3:57 pm [\u0026#8230;] Castleberry Hill Historic District Located just southwest of the city and loosely bounded by McDonald Street, Peters Street, Walker Street, and Northside Drive, the Castleberry Hill Historic District lies just across the railroad tracks from both the Georgia World Congress Center and Mercedes-Benz Stadium. Formerly an industrial hub of warehouses and factories, the district\u0026#8217;s history dates back to the late 19th century when it was an ideal site for railroad operations and warehouse storage. Today, Castleberry Hill has transformed into an art-focused district, where the old warehouses have been repurposed as art galleries, studios, boutiques, and creative spaces. Once a month, the district comes alive with art walks and other events, attracting art lovers from across Atlanta. [19]. [\u0026#8230;]\r\n\r\n## Primary Sidebar\r\n\r\n### Stay in the know!\r\n\r\nDo you suffer from FOMO, that pervasive fear of missing out? We can help.\r\n\r\nSign up for email updates from ATLANTA ON THE CHEAP and never miss a great deal or free event.\r\n\r\n### old google site search bar\r\n\r\n## Secondary Sidebar\r\n\r\n### Local Groupon Deals\r\n\r\n[](https://www.tkqlhce.com/click-100651436-15245379)### Groupon Experiences\r\n\r\n[](https://www.tkqlhce.com/click-100635924-15245432?sid=ATL)### See something you like?\r\n\r\nPlease tell your friends about it! We hope you\u0026#8217;ll like, share, follow, and spread the word about Atlanta On The Cheap!\r\n\r\n\r\n\r\n### Like. Follow.\r\n\r\n[](https://www.facebook.com/atlantaonthecheap)[](https://twitter.com/ATLcheap)### We use stock images by our partner, Deposit Photos!\r\n\r\n\u0026nbsp;\r\n\r\n\r\n\r\n**www.depositphotos.com**\u0026nbsp;\r\n\r\n### Member of the Living on the Cheap Network\r\n\r\n[](http://livingonthecheap.com/our-network/)Atlanta on the Cheap is a member of Living on the Cheap, a network of websites published by journalists and consumer advocates.[](https://livingonthecheap.com/)* Back to Top\r\n* Half-Off Deals\r\n* Free\r\n* Kids \u0026#038; Family\r\n* Events\r\n* Attractions\r\n* Concerts\r\n* Theater\r\n* Sports\r\n* Terms of Use and Privacy Policy\r\n\r\n\u0026#x000A9;\u0026nbsp;2024 Atlanta on the Cheap. Living On The Cheap\u0026reg; and On the Cheap\u0026reg; are registered trademarks owned by Living On The Cheap Inc. Site design by Five J\u0027s Design.All the stories on our site are protected by copyright and any unauthorized copying is strictly prohibited. Plus, it is just not nice.\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in October in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading\u0022The Lion King\u0022 (Photo by Joan Marcus)\r\n\r\n# Cool Things To Do in October in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished September 13, 2024Last Updated September 26, 2024\r\n\r\nThe weather has cooled off and Atlanta is ready to embrace the fall with new exhibits, fashion shows and festivals \u2014 lots and lots of festivals. October always brings Atlanta Pride. Prepare with our Pride Survival Guide. Hispanic Heritage Month continues through Oct. 15.\r\n\r\n## The Lion King\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1706,c_scale/v1726166160/newAtlanta.com/S9-GeraldRamseyasMufasa.TheLionKingNorthAmericanTourC2A9Disney.PhotobyMatthewMurphy28229/S9-GeraldRamseyasMufasa.TheLionKingNorthAmericanTourC2A9Disney.PhotobyMatthewMurphy28229.jpg?_i=AA)\r\nGerald Ramsey portrays Mufasa in \u201CThe Lion King\u201D National Tour. (Photo by Matthew Murphy)The Regions Bank Broadway in Atlanta series starts up again and leads with the very popular \u201CThe Lion King\u201D Oct. 2 through Oct. 20 at the Fox Theatre. \u201CThe Lion King\u201D toured North America for more than 22 years, and during that time welcomed more than 23 million theater goers, making it North America\u2019s longest-running and most-attended Broadway tour. This lavish production features a score by Elton John, Tim Rice and additional songs by Lego M, Mark Mancina, Jay Rifkin, Julie Taylor and Hans Zimmer. \u201CThe Lion King\u201D is sure to dazzle theater lovers of all ages.\r\n\r\n## Jazz it Up\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166194/newAtlanta.com/Hammonds-House-Museum-photo-credit-Karen-Hatchett-copy/Hammonds-House-Museum-photo-credit-Karen-Hatchett-copy.jpg?_i=AA)\r\nGet jazzed at Hammonds House Museum. (Photo by Karen Hatchett)An exciting weekend of jazz will take place at the Hammonds House Museum as trombonist Wycliffe Garden and pianist Eric Scott Reed perform Oct. 5 and present a workshop for musicians and jazz lovers on Oct. 6. A veteran member of the Wynton Marsalis Septet and an original member of Lincoln Center Jazz Orchestra, Gordon has had an impressive career touring the world performing hard-swinging, straight-ahead jazz\u0026nbsp;while receiving great acclaim from audiences and critics alike. Eric Scott Reed has been a vital force in jazz for more than 25 years, excelling as a leader, composer, producer, educator, sideman and member of\u0026nbsp;the Marsalis\u2019 Septet and Lincoln Center Jazz Orchestra.\r\n\r\n## Get a Room\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166250/newAtlanta.com/Hotel-Granada_27893b8e55/Hotel-Granada_27893b8e55.jpg?_i=AA)\r\nThe reimagined Granada Hotel sits just across from the Woodruff Arts Center. (Photo by Grady McGill)Atlanta has amazing hotel rooms \u2014 something for everyone \u2014 and a few more hotels just opened. Hotel Granada, across from the Arts Center MARTA station and the Woodruff Arts Center, was built in 1923 and fell on some hard times. But it has been lovingly reimaged with contemporary Southern flair. At the heart of the 120-room hotel is Pom Court, an open-air oasis that is perfect for hotel guests, locals and museum goers (the High Museum of Art is nearby) to sit on the patio and enjoy a cocktail or quick bite.\r\n\r\nThe Forth Hotel just opened by the Beltline in the Historic Fourth Ward. The hotel features 157 guest rooms and suites as well as 39 residential suites, elevated restaurants and bars. Guests will enjoy an outdoor pool and state-of-the-art wellness amenities, including a 20,000-square-foot fitness center, sauna, steam room, leg compression sleeves, massage guns and vibrating massage balls, and foam rollers.\r\n\r\nIn November, the Origin Hotel Atlanta will welcome guests to its \u201950s-era office building-turned-boutique hotel. The hotel features 122 guest rooms and suites plus Butter \u002B Scotch restaurant and bar, a local retail store front and event space. The Wyndham Hotel is located on Pryor Street, just steps away from the Georgia World Congress Center, Mercedes-Benz Stadium and Centennial Olympic Park.\r\n\r\n## There\u2019s Renoir, Degas, Monet \u2014 and More\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1440,c_scale/v1726166729/newAtlanta.com/Interior-first-exhibition-\u00C2\u00A9-Excurio-GEDEON-Experiences-Musee-dOrsay-copy/Interior-first-exhibition-\u00C2\u00A9-Excurio-GEDEON-Experiences-Musee-dOrsay-copy.jpg?_i=AA)\r\nTravel back to the 19th century at the Eclipse Centre. (Photo courtesy of Excurio-GEDEON Experiences Mus\u00C3\u00A9e)\u201CTonight with the Impressionists\u201D is a virtual reality exhibit coming to the Eclipse Centre that will take you back to 1874 to meet the famous painters of that period. Created in collaboration with the renowned Mus\u00E9e d\u2019Orsay in Paris and in honor of the 150th anniversary of the first-ever Impressionist exhibition, the 45-minute journey transports you back to 1874 when a group of artists broke away from traditional academic painting, focusing instead on capturing light, color and atmosphere in new ways. Known as the Impressionists\u2014Monet, Renoir, Morisot, Degas and others\u2014they depicted everyday life and outdoor scenes with spontaneous brushstrokes and vibrant colors. The exhibition runs through December.\r\n\r\n## Little Five Points Parade\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)\r\nYou might have way too much fun at the Little Five Points Halloween Festival and Parade. (Photo by Reid Koski)Yes, we\u2019ve all seen Halloween parades, but you really haven\u2019t seen a Halloween parade until you venture down to Little Five Points. The bohemian neighborhood sponsors a Halloween Festival on Oct. 19 and 20 that features live music from the city\u2019s best underground acts, an art and vintage market, ghost tours, a skate park, and food and drink vendors. But it\u2019s the parade \u2014 the biggest and spookiest in the Southeast \u2014 that makes it a signature fall event. Don\u2019t believe us \u2014 a number of national publications such as Country Living and Fodors \u2014 have named it among the best Halloween parades in the country. For more Halloween happenings in Atlanta check our Spooktacular Guide to Halloween in Atlanta and our Grown-Up Guide to Halloween in Atlanta.\r\n\r\n## Experience \u2018Fantaween\u2019 at Pemberton Place\r\n\r\nExperience \u201CFantaween,\u201D in Centennial Olympic Park, Saturdays and Sundays Oct. 5-27, from 10 a.m. until 7 p.m. You will journey through haunted grounds filled with eerie illusions and spooky surprises. You\u2019ll find photo ops around every corner, and you may encounter a ghostly friend. Visit Georgia Aquarium and World of Coca-Cola in order to enjoy a limited-edition spooky FANTA Frightful Fizz beverage served in a collectible cup from Bottle Cap Cafe. This is a ticketed event.\r\n\r\n## Dragons and the Atlanta Symphony Orchestra\r\n\r\nDreamWorks\u2019 \u201CHow to Train a Dragon\u201D comes to the Woodruff Arts Center Oct. 4 and 5, with a captivating and original story that combines humor, fire-breathing action and epic adventure.\u0026nbsp; Hiccup is a young Viking who defies tradition when he befriends one of his deadliest foes \u2013a ferocious dragon he calls Toothless. Together, these unlikely heroes must fight against all odds to save both their worlds.\r\n\r\n\u201CHow To Train Your Dragon\u201D\u2013In Concert features this acclaimed film presented in HD, with composer John Powell\u2019s Academy Award-nominated score performed live to picture by a full symphony orchestra.\r\n\r\nAlso, don\u2019t miss the Atlanta Symphony Orchestra\u2019s performances of Haydn and Mendelssohn Symphonies Oct. 17-19 and the return Georgia native Roderick Cox who will dig into the rich sounds of the ASO string section for Barber\u2019s soul-stirring \u201CAdagio.\u201D He follows with music from John Adams\u2019 white-knuckle opera\u0026nbsp;\u201CDoctor Atomic\u201D and concludes with the Atlanta Symphony Orchestra going full throttle in Rachmaninoff\u2019s seductive orchestral romp\u0026nbsp;\u201CSymphonic Dances.\u201D\r\n\r\n## Let\u2019s Brunch a Bunch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166331/newAtlanta.com/Ela-Brunch/Ela-Brunch.jpg?_i=AA)\r\nPut a Mediterranean twist on your brunch at Ela. (Photo by Grady McGill)The weather is cooling off so it\u2019s a perfect time for brunching with friends. Several restaurants are offering brunch options. Ela offers a Mediterranean twist with Turkish hash or souvlaki and eggs. Snooze offers a fall menu that includes pumpkin pecan pie pancakes, pumpkin martinis and a jalape\u00F1o and cheddar omelet.\r\n\r\nIndaco started a weekend brunch with dishes such as breakfast pizza, ricotta toast and eggs in purgatory. The Flying Biscuit always offers a great bunch menu; fall items include cinnamon roll pancakes and a bacon jam burger bowl.\r\n\r\n## Stylin\u2019\r\n\r\nAtlanta Fashion Week (ATLFW) makes a highly anticipated return Oct. 2-7, bringing together a global array of designers and brands to celebrate fashion, art, culture and creativity. This year\u2019s ATLFW promises to be a truly international affair, showcasing renowned talents from Nigeria, the United Kingdom, Senegal and New York, to name a few. Throughout the week, fashion will take center stage for immersive experiences, complete with pop-up showroom displays, brand activations and runway shows.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in September in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n\r\n## Corn Dogs, Rides and Whack a Mole\r\n\r\nThe Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.\r\n\r\n## Pickleball Tournament\r\n\r\nIt\u2019s Pickle Palooza. The Collective Learning Academy of Atlanta is sponsoring a fund-raiser that promises an evening of pickleball action for players of all levels. Taking place Oct. 1 at the Atlanta Pickleball Center, the round robin matches will begin at 6 and go until the knockout round around 9 p.m. There will be light appetizers and beverages. Want to get tournament ready? Head over to Painted Pickle for a few practice rounds.\r\n\r\n## Nando\u2019s African Diaspora Art Comes to Atlanta\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1815,h_2560,c_scale/v1726166411/newAtlanta.com/Fisani-Parktown-by-Zenele-Muholi-copy/Fisani-Parktown-by-Zenele-Muholi-copy.jpg?_i=AA)\r\nFisani, Parktown by Zenele MuholiA world-class exhibition curated from\u0026nbsp;Nando\u2019s art collection, \u201CIf You Look Hard Enough, You Can See Our Future,\u201D is making a stop in Atlanta. The world-class exhibition will feature nearly 30 pieces at the African Diaspora Art Museum of Atlanta (ADAMA).\r\n\r\nOpen Sept. 20 through Nov. 2, the exhibition will feature the works of African artists including Zanele Muholi,\u0026nbsp;Portia Zvahera\u0026nbsp;and\u0026nbsp;Mbongeni Buthelizi.\u0026nbsp;\r\n\r\nNando\u2019s, whose restaurants serve South-African flamed-grilled Peri-Peri chicken and Peri-Peri sauce,\u0026nbsp;owns one of the world\u2019s largest collections of Southern African contemporary art and design. The work is displayed in its restaurants globally, including in Atlanta.\r\n\r\n## Holographic Theater Comes to Ponce City Market\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166509/newAtlanta.com/Verse-Immersive-Headset_Photo-Courtesy-of-Verse-Immersive-resized/Verse-Immersive-Headset_Photo-Courtesy-of-Verse-Immersive-resized.png?_i=AA)\r\nTry an augmented reality experience at Verse Immersive. (Photo courtesy of Verse Immersive)A new augmented reality experience, Verse Immersive, is open at The Roof at Ponce City Market. The only holographic theater of its kind, Verse Immersive uses AR visors to bring 3D characters and stories to life. Verse Immersive offers guests an immersive experience of interactive tales. Guests can expect a 30-, 60-, or 90-minute immersive journey during which AR visors will put them at the center of an interactive and engaging story. AR, unlike virtual reality, allows guests to roam freely throughout the room while still seeing and interacting with their surroundings. Five experiences are offered at Verse Immersive and all are appropriate for ages 7 and older.\r\n\r\n## Festivals, Festivals and More Festivals\r\n\r\nFall is the perfect time to explore neighborhoods, hear local musicians, purchase the works of artists and have fun. Yes, it\u2019s festival time and no matter which direction you go, there\u2019s a festival. Among our favorites are the Fall Festival on Ponce, Candler Park Fall Festival, Old Fourth Ward Arts Fest and the Scottish Festival and Highland Games at Stone Mountain Park. ONE MusicFest turns the sound up in Piedmont Park Oct. 26 and 27 with its brand of urban progressive music. Among the performers will be Cardi B, Earth, Wind and Fire, T.I. and Keyshia Cole. OYE Festival on Oct. 6 provides an opportunity to continue celebrating Hispanic Heritage Month. It is a day filled with music, art installations, food and interactive experiences in the heart of Atlanta in Old Fourth Ward. Oct. 13 brings Latinx Art Exhibit to the Decatur Arts Alliance Gallery.\r\n\r\n## A Gamer\u2019s Dream\r\n\r\nDreamHack comes to Atlanta Oct. 4-6 with a weekend full of esports, cosplay, creator-led activities, table top, special guests, panels, workshops, IRL loot stores and, oh yes, gaming. Among the fun will be Brennan Lee Mulligan\u2019s D\u0026amp;D One Shot, the Call of Duty: Mobile World Championship and a cosplay contest.\r\n\r\n## Happy Anniversary\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166637/newAtlanta.com/IMG_0701/IMG_0701.jpg?_i=AA)\r\nMmmm. Munch a grilled skewer of tasty bites at Yakitori Kona. (Photo by Grady McGill)Let\u2019s blow out the candles as several of our favorite hotels and restaurants celebrate another year of pleasing visitors and locals with great food and hospitality. Celebrating first years are Yakitori Kona, an intimate izakaya cocktail bar offering grilled skewers in Virginia Highland, and Verdure, a Midtown restaurant that fuses the flavors and diverse cooking techniques of western and African countries.\r\n\r\nKrog Street Market, the acclaimed food court, celebrates its 10th anniversary while the Hotel Clermont takes the cake with its 100th anniversary. Here\u2019s to many more.\r\n\r\n## Light up Cancer\r\n\r\nThe Leukemia \u0026amp; Lymphoma Society\u2019s Light The Night brings light to the darkness of cancer by supporting lifesaving research, advocacy and services for people battling blood cancer. Join family and friends on Oct. 5 at Piedmont Park to celebrate, honor or remember those affected by blood cancers. Registration is free, but participants are encouraged to raise funds to support the mission. All registered participants will receive an illuminated lantern. In addition to the walk, the evening will be filled with music, fireworks, a kid zone and empowering ceremonies honoring survivors and remembering those lost.\r\n\r\n## Take a Stroll\r\n\r\nAtlanta Streets Alive welcomes walkers, skaters, bicycle riders and cartwheel turners to Ralph David Abernathy Boulevard and Georgia Avenue from 1 until 5 p.m. Oct. 20 when the streets will be closed to vehicular traffic and open to pedestrians. Walk on, Atlanta.\r\n\r\n## It\u2019s Fall: Time for College Football\r\n\r\nFall sports mean one thing in particular in the South: college football. Whatever your alma mater, chances are there\u2019s a bar in Atlanta where you can join like-minded fans to cheer on your team. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Fashion Forward\r\n\r\nFashionFest, which takes place Oct. 11-13, is an immersive ever-evolving platform giving attendees an insider\u2019s look into the world of urban and pop culture by combining elements of a festival and an exhibition and bringing together diverse aspects of the business fashion world.\r\n\r\nFashionFest offers attendees the opportunity to engage with and learn from some of the most fascinating brands and business minds in urban and pop fashion culture, while enjoying live musical performances, exploring the curated marketplace to shop limited edition goods and experiencing cutting-edge technology and sourcing while learning business principles to grow and expand your fashion brands.\r\n\r\nTo find things to do in Atlanta any time, visit our calendar of events.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Arts \u0026amp; CultureSeptember is a great time to be in Atlanta. There are amazing festivals \u2014 food and music \u2014 Black Pride events and football. Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Arts \u0026amp; CultureFrom outdoor activities to holiday shows, there\u2019s something cool for everyone during December in Atlanta. Cool Things To Do in December in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

Here are some interesting events happening in Atlanta for each of the next four weekends:

### Weekend of October 5-6, 2024
1. **Festival on Ponce** - An arts festival in Olmsted Linear Park with over 125 vendors of fine art, food, drinks, and children’s activities.
2. **Candler Park Fall Fest** - A popular annual festival with live music, art, food, rides, games, and more in Candler Park.
3. **Atlanta Horror Film Festival** - A festival of horror films at the Limelight Theater.
4. **Taste of Tucker** - A food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music, and more.

### Weekend of October 12-13, 2024
1. **Atlanta Pride Festival** - One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace, and more.
2. **Old Fourth Ward Fall Arts Festival** - Featuring local and Georgian artists, demonstrations, live entertainment, art activities, and food in Historic Fourth Ward Park.
3. **Chalktoberfest** - An annual art and beer festival in Marietta Square, featuring professional chalk artists, craft beers, live music, arts and crafts, and food trucks.
4. **Joyce Meyer Live** - A faith event at the Gas South Arena.

### Weekend of October 19-20, 2024
1. **Little 5 Points Halloween Festival & Parade** - A popular annual Halloween celebration with live music, parade, haunted house, ghost tours, and more.
2. **PNC Atlanta 10 Miler & 5K** - Annual runs through Buckhead, starting from Atlantic Station.
3. **Georgia Tech vs. Notre Dame** - A thrilling college football game at Mercedes-Benz Stadium.
4. **Stone Mountain Highland Games and Scottish Festival** - A festival celebrating Scottish heritage at Stone Mountain Park with traditional Scottish athletics, music, dancing, food, and drink.

### Weekend of October 26-27, 2024
1. **ONE Musicfest** - A big annual festival of urban progressive music in Piedmont Park.
2. **Boo at the Zoo** - A Halloween event at Zoo Atlanta with trick or treating, Halloween-themed activities, and more.
3. **Sneaker Con Atlanta** - A sneaker convention at Cobb Galleria Centre celebrating its 15th anniversary with a bigger and better show.
4. **Air Dot Show Tour** - An air show at Atlanta Regional Airport, Peachtree City.

For more details and a comprehensive list of events, you can visit:
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
- [Atlanta GA Events](https://atlanta-ga.events/october/)
- [Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

![Festival on Ponce](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)

Enjoy your weekends in Atlanta!


In [7]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here are some interesting events happening in Atlanta for each of the next four weekends:

### Weekend of October 5-6, 2024
1. **Festival on Ponce** - An arts festival in Olmsted Linear Park with over 125 vendors of fine art, food, drinks, and children’s activities.
2. **Candler Park Fall Fest** - A popular annual festival with live music, art, food, rides, games, and more in Candler Park.
3. **Atlanta Horror Film Festival** - A festival of horror films at the Limelight Theater.
4. **Taste of Tucker** - A food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music, and more.

### Weekend of October 12-13, 2024
1. **Atlanta Pride Festival** - One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace, and more.
2. **Old Fourth Ward Fall Arts Festival** - Featuring local and Georgian artists, demonstrations, live entertainment, art activities, and food in Historic Fourth Ward Park.
3. **Chalktoberfest** - An annual art and beer festival in Marietta Square, featuring professional chalk artists, craft beers, live music, arts and crafts, and food trucks.
4. **Joyce Meyer Live** - A faith event at the Gas South Arena.

### Weekend of October 19-20, 2024
1. **Little 5 Points Halloween Festival & Parade** - A popular annual Halloween celebration with live music, parade, haunted house, ghost tours, and more.
2. **PNC Atlanta 10 Miler & 5K** - Annual runs through Buckhead, starting from Atlantic Station.
3. **Georgia Tech vs. Notre Dame** - A thrilling college football game at Mercedes-Benz Stadium.
4. **Stone Mountain Highland Games and Scottish Festival** - A festival celebrating Scottish heritage at Stone Mountain Park with traditional Scottish athletics, music, dancing, food, and drink.

### Weekend of October 26-27, 2024
1. **ONE Musicfest** - A big annual festival of urban progressive music in Piedmont Park.
2. **Boo at the Zoo** - A Halloween event at Zoo Atlanta with trick or treating, Halloween-themed activities, and more.
3. **Sneaker Con Atlanta** - A sneaker convention at Cobb Galleria Centre celebrating its 15th anniversary with a bigger and better show.
4. **Air Dot Show Tour** - An air show at Atlanta Regional Airport, Peachtree City.

For more details and a comprehensive list of events, you can visit:
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
- [Atlanta GA Events](https://atlanta-ga.events/october/)
- [Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

![Festival on Ponce](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)

Enjoy your weekends in Atlanta!

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
